
# Import dữ liệu FD Balance mới vào file lưu trữ

In [62]:
import pandas as pd
import numpy as np
from os import path,getcwd
import pickle as pickle


In [63]:
import pandas as pd
import numpy as np
from os import path,getcwd
# import pickle5 as pickle

def change_type_table(file):
    file.ACCTNO=file.ACCTNO.astype("uint64")
    file.CIFNO=file.CIFNO.astype("uint32")
    file.BRN=file.BRN.astype("uint16")
    file.STATUS=file.STATUS.astype("uint8")
    file.CDTERM=file.CDTERM.astype("uint16")
    file.CDTCOD=file.CDTCOD.astype('category')
    file.RATE=file.RATE.astype('float32')
    file.CURTYP=pd.Categorical(file.CURTYP, categories=['AUD','BND','CAD','CHF','CNY','CZK','DKK','EUR','FJD','GBP','HKD','IDR','INR','JPY','KHR', 'KRW', 'LAK', 'LKR', 'MMK', 'MYR', 'NOK', 'NZD', 'PHP', 'PLN', 'RUB', 'SEK', 'SGD', 'THB', 'TRY', 'TWD', 'USD', 'VND', 'XAU'])

# path
code_folder=getcwd()
output_folder=path.join(code_folder,'Output')
data_folder =path.join(code_folder,'Data')
fd_save = path.join(code_folder,'fd_save')
fd_import_file = path.join(data_folder, 'fd_files')
# gán biến parameter
end_date=pd.Timestamp('2021-08-31')
start_date=pd.Timestamp('2015-10-31')
list_bank=['L','S','B','C','R',"F","NF"]
list_CCY=['VND','USD','OTH']
tw = [1,7,30,90,180,365,730,1095,1460,1825,2190]
list_date=pd.date_range(start=start_date,end=end_date).tolist()

file_data_name=path.join(data_folder,"FD_BALANCE_30062022.h5")
store=pd.HDFStore(file_data_name)

# gán biến to import data

store.open()
to_date=pd.Timestamp('2022-08-31')
from_date=store.select_column("data","CHECK_TO_DATE").max()+pd.Timedelta("1D")
list_date_import=pd.date_range(start=from_date,end=to_date).tolist()

for import_day in list_date_import[:2]:
    print(import_day,end='\r')
    store.open()
    file_name="CD_{}".format(import_day.strftime(format="%Y_%m_%d"))
    try:
        #data=pickle.load(open(path.join(fd_import_file,"{}.pickle".format(file_name.upper())),"rb"))
        #data = pd.read_csv(path.join(fd_import_file, "{}.csv".format(file_name)))
        data = pd.read_csv(r"D:\IRRBB 2022\Data\fd_files\{}.csv".format(file_name))
        if data['MAT_DATE'].dtypes == np.dtype('O'):
            data['MAT_DATE'] = pd.to_datetime(data['MAT_DATE'])
        if data['OPEN_DATE'].dtypes == np.dtype('O'):
            data['OPEN_DATE'] = pd.to_datetime(data['OPEN_DATE'])
        data.rename(columns={'{}'.format(import_day):"CURRENT_BALANCE"},inplace=True)
        data.drop('Unnamed: 0', axis = 1, inplace = True)
    except:
        data=pd.read_sas(path.join(fd_import_file,"{}.sas7bdat".format(file_name)), encoding='iso-8859-1')
        current_balance="CURRENT_BALANCE" if "CURRENT_BALANCE" in data.columns else file_name.upper()
        data.rename(columns={current_balance:"CURRENT_BALANCE"},inplace=True)
    change_type_table(data)
    data["CHECK_TO_DATE_1"]=import_day
    data["CHECK_FROM_DATE_1"]=import_day
    raw=store.select('data','CHECK_TO_DATE==\'{}\''.format((import_day-pd.Timedelta("1D")).strftime(format="%Y-%m-%d")))
    raw=raw.merge(data,how="outer",indicator=True)
    raw.loc[raw._merge!="left_only","CHECK_TO_DATE"]=raw.loc[raw._merge!="left_only","CHECK_TO_DATE_1"]
    raw.loc[raw._merge=="right_only",'CHECK_FROM_DATE']=raw.loc[raw._merge=="right_only",'CHECK_FROM_DATE_1']
    raw.drop(['CHECK_FROM_DATE_1','CHECK_TO_DATE_1',"_merge"],axis=1,inplace=True)
    change_type_table(raw)
    store.remove("data",'CHECK_TO_DATE==\'{}\''.format((import_day-pd.Timedelta("1D")).strftime(format="%Y-%m-%d")))
    store.put('data', raw, format='table', data_columns=True,index=False,append=True)
    # store.close()
print("Done!")

KeyError: 'No object named data in the file'

# Chạy Mô hình FD 

In [65]:
import pandas as pd
import numpy as np
from os import path,getcwd
from IPython.display import clear_output

In [66]:
# FUNCTION
def classify_term_func(x):
    if x <1 : m="<1M"
    elif x<3 : m="1-3M"
    elif x<6 : m="3-6M"
    elif x<9 : m="6-9M"
    elif x<12: m="9-12M"
    else: m=">12M"
    return m

def modify_columns_name(table,level,name,axis=1):
    if axis==0:
        table=table.T.copy()
    else:
        table=table.copy()
    table.columns=pd.MultiIndex.from_tuples([tuple(x[:level])+tuple([name])+tuple(x[level+1:]) for x in table.columns],names=table.columns.names)
    if axis==0: table=table.T
    return table

def Calculate_for_merge_bank(table,level_of_bank=1,return_merged_bank_only=False):
    """
    Tính toán cho RC, LSB, BW_excl_F_NF, BW
    """
    table=table.copy()
    L_table=table[table.columns[table.columns.get_level_values(level_of_bank)=="L"]]
    S_table=table[table.columns[table.columns.get_level_values(level_of_bank)=="S"]]
    B_table=table[table.columns[table.columns.get_level_values(level_of_bank)=="B"]]
    R_table=table[table.columns[table.columns.get_level_values(level_of_bank)=="R"]]
    C_table=table[table.columns[table.columns.get_level_values(level_of_bank)=="C"]]
    F_table=table[table.columns[table.columns.get_level_values(level_of_bank)=="F"]]
    NF_table=table[table.columns[table.columns.get_level_values(level_of_bank)=="NF"]]
    
    SB_table=modify_columns_name(S_table,1,"SB").fillna(0)\
    .add(modify_columns_name(B_table,1,"SB"),fill_value=0)

    LSB_table=modify_columns_name(S_table,1,"LSB").fillna(0)\
    .add(modify_columns_name(B_table,1,"LSB"),fill_value=0)\
    .add(modify_columns_name(L_table,1,"LSB"),fill_value=0)

    RC_table=modify_columns_name(R_table,1,"RC").fillna(0)\
    .add(modify_columns_name(C_table,1,"RC"),fill_value=0)

    BWexcl_table=modify_columns_name(RC_table,1,"BWexcl").fillna(0)\
    .add(modify_columns_name(LSB_table,1,"BWexcl"),fill_value=0)

    BW_table=modify_columns_name(BWexcl_table,1,"BW").fillna(0)\
    .add(modify_columns_name(F_table,1,"BW"),fill_value=0)\
    .add(modify_columns_name(NF_table,1,"BW"),fill_value=0)

    table=pd.concat([table,RC_table,SB_table,LSB_table,BWexcl_table,BW_table],axis=1)
    
    columns_table=table.columns
    if return_merged_bank_only:
        columns_table=columns_table[columns_table.get_level_values(0).isin(["VND","USD"]) 
                                    & columns_table.get_level_values(level_of_bank).isin(["RC","LSB","BWexcl","SB"])]
        
    reindex=columns_table[columns_table.get_level_values(0)=="VND"].to_list()\
        +columns_table[columns_table.get_level_values(0)=="USD"].to_list()\
        +columns_table[columns_table.get_level_values(0)=="OTHER"].to_list()
    
    return table[reindex].replace(0,np.nan).copy()

def Calculate_ALL_TERM(table,level_Classify_term,axis=0):
    total_level=list(range(table.index.nlevels))
    table=modify_columns_name(table,level_Classify_term,"ALL_TERM",axis=axis)
    return table.groupby(level=total_level).sum()

def handling_data(data_input,bank_input,sfs_input,list_date_input,cal_cbal=True):
    data=data_input.copy()
    # Classify term
    data["Classify_term"]=(data.CDTCOD.astype(str).map(lambda x : 1 if x=="M" else 1/30)*data.CDTERM).map(classify_term_func)
    # quy các đồng tiền khác USD về VND, đổi tên thành "OTHER"
    data["CURRENT_BALANCE"]=data[["CURRENT_BALANCE"]].to_numpy()*sfs_input.set_index("CURRENCY_CODE").reindex(data.CURTYP).to_numpy()
    data.CURTYP=data.CURTYP.astype("str")
    data.loc[data.CURTYP.isin(["VND","USD"])==False,"CURTYP"]="OTHER"
    # update the newest CIF for account (có những acc chuyển CIF) 
    data=data.sort_values(["ACCTNO","CHECK_FROM_DATE"]).reset_index(drop=True)
    data.loc[data.duplicated(["ACCTNO"],keep="last"),"CIFNO"]=np.nan
    data.CIFNO.fillna(method="bfill",inplace=True)
    # merge bank
    data=data.merge(bank_input.rename(columns={"cif_number":"CIFNO"}),how="left")

    if cal_cbal:
        # thêm cột cbal từng ngày
        data[list_date_input]=np.array([np.where((data.OPEN_DATE>i),np.nan,np.array((((data.CHECK_FROM_DATE<=i) & (data.CHECK_TO_DATE>=i))*(data.CURRENT_BALANCE)))) for i in list_date_input]).T
        # group by theo account
        data=data.groupby(["ACCTNO","bank","CURTYP","Classify_term"],as_index=False).agg({**{"MAT_DATE":np.max},**{n: np.sum for n in list_date_input}},skipna=True)
        # drop những account có cbal theo chuỗi quan sát đều bằng 0
        data.drop(data[np.nansum(data[list_date_input],1)==0].index,inplace=True)
        # thêm cột ngày cuối cùng có số dư > 0
        data["LAST_DAY_CBAL>0"]=[list_date_input[::-1][i.index(1)] for i in (data[list_date_input[::-1]]!=0).astype(int).values.tolist()]
        # thêm cột số dư tại ngày cuối cùng có số dư > 0
        data["CBAL_AT_LAST_DAY_CBAL>0"]=[data.loc[i,data.loc[i,"LAST_DAY_CBAL>0"]] for i in data.index]
        # thêm cột trạng thái account là rút trước hạn (pre) hay đúng hạn (on) hay không xác định/chưa rút (unidentified)
        data["WD_STT"]=((data["LAST_DAY_CBAL>0"]+pd.Timedelta("1D"))<data.MAT_DATE).map(lambda x: "pre" if x else "on")
        data.loc[data[end_date]>0,"WD_STT"]="unidentified"
        data.WD_STT=pd.Categorical(data.WD_STT)
    return data

def bank_handle(path):
    bank=pd.read_csv(path)
    try:
        bank.loc[(bank.CUSTOMER_RATING=="41") & (bank.bank=="F"),"bank"]="NF" 
    except:
        pass
#         print( "--- loi---",link)
    bank=bank.rename(columns={("cifnew" if "cifnew" in bank.columns else "cif_number"):"cif_number"})[["cif_number","bank"]]
    return bank.copy()

In [67]:
# path
code_folder=getcwd()
output_folder=path.join(code_folder,'Output')
data_folder =path.join(code_folder,'Data')

In [68]:
# gán biến parameter
end_date=pd.Timestamp('2022-08-31') # ngày cuối cùng cả train +test model
start_date=pd.Timestamp('2015-10-31')
list_bank=['L','S','B','C','R',"F","NF"]
list_CCY=['VND','USD','OTHER']
tw_list = ['0','1','7','30','90','180','365','730','1095','1460','1825','2190']
classify_term=["<1M","1-3M","3-6M","6-9M","9-12M",">12M","ALL_TERM"]
kind=["WD","PRE"]

# tạo biến trung gian
list_date=pd.date_range(start=start_date,end=end_date).tolist()
multi_index=np.array([[[[[(c,b,ct,t,k) for k in kind ] for t in tw_list] for ct in classify_term] for b in list_bank] for c in list_CCY]).reshape(-1,5)
multi_index=pd.MultiIndex.from_frame(pd.DataFrame(multi_index,columns=["CURTYP","bank","Classify_term","tw","Kind"]))

# Khai báo seasonal
Solar=[pd.Timestamp(year=y,month=1,day=1)for y in range(2016,2025)]
Lunar=[pd.to_datetime(i,dayfirst=True) for i in ['8/2/2016', '28/1/2017', '16/2/2018', '5/2/2019', '25/1/2020','12/02/2021','01/02/2022','22/01/2023',"10/02/2024"]]
ss_day_t0=[x for y in [pd.date_range(Solar[i]-pd.Timedelta("45D"),Lunar[i]-pd.Timedelta("30D")) for i in range(len(Solar))] for x in y]

In [298]:
check_rb_usd = pd.read_csv(r"C:\Users\ngocna3\Documents\My Received Files\Work\CD_IRRBB\FD_2022\Output\wd_ratio_310822.csv", header = [0,1,2,3], index_col = [0])

In [299]:
if check_rb_usd.index.dtype == np.dtype("O"):
    check_rb_usd.index = pd.to_datetime(check_rb_usd.index)

In [300]:
check_rb_usd = check_rb_usd[(check_rb_usd.index.isin(ss_day_t0))]

In [301]:
check = check_rb_usd.loc[:,(check_rb_usd.columns.get_level_values(0) == 'USD',check_rb_usd.columns.get_level_values(1) == 'RC',check_rb_usd.columns.get_level_values(3) == '730')]

In [309]:
check.index

DatetimeIndex(['2015-12-11', '2015-11-17', '2015-11-18', '2015-11-19',
               '2015-11-20', '2015-11-21', '2015-11-22', '2015-11-23',
               '2015-11-24', '2015-11-25',
               ...
               '2022-01-01', '2022-12-01', '2022-01-02', '2022-12-02',
               '2022-12-03', '2022-12-04', '2022-12-05', '2022-12-06',
               '2022-12-07', '2022-12-08'],
              dtype='datetime64[ns]', length=353, freq=None)

In [316]:
check[(check.index == pd.Timestamp('2016-01-05'))]

CURTYP,USD
bank,RC
KIND,WD_PRE_RATIO
TW,730
2016-01-05,1.0


In [279]:
check

CURTYP      USD     
bank         RC     
KIND        ALL  PRE
TW          730  730
2015-11-17  0.0  0.0
2015-11-18  0.0  0.0
2015-11-19  0.0  0.0
2015-11-20  0.0  0.0
2015-11-21  0.0  0.0
...         ...  ...
2021-12-29  NaN  NaN
2021-12-30  NaN  NaN
2021-12-31  NaN  NaN
2022-01-01  NaN  NaN
2022-01-02  NaN  NaN

[355 rows x 2 columns]

In [74]:
bank_train = bank_handle(path.join(data_folder, 'bank_2022_07_31_train.csv'))
bank_test = bank_handle(path.join(data_folder, 'bank_2022_08_31_train+test.csv'))
bank=bank_train.combine_first(bank_test)
'''check NF F
Do bang bank bi gop NF + F = F, xin them file ben RDM de boc tach NF va F
'''
NFI = pd.read_excel(path.join(data_folder, 'F_NF 31082022.xlsx'))
NF_acc = NFI[(NFI.INDICATOR_BU == 'NF')]
list_NF = NF_acc.CPTY_ID_CORE.unique().tolist()
bank.loc[(bank.cif_number.isin(list_NF)), 'bank'] = 'NF'
bank.to_csv(path.join(data_folder,'final_bank_310822.csv'))
#bank = bank_handle(path.join(data_folder,'bank_2021_10_31.pickle'))
#bank.to_pickle(path.join(data_folder,r"Data_final_for_2022\bank_total_2021_10_31.pickle"))

In [75]:
# ty gia
sfs=pd.read_excel(path.join(data_folder,'SSFXRT_MHCC_2022_08_31_train+test.xlsx'))
sfs.columns=sfs.columns.map(lambda x:x.upper())
sfs.loc[sfs.CURRENCY_CODE=="USD","MID_RATE"]=1

In [76]:
sfs.drop('UNNAMED: 0', axis = 1, inplace = True)

In [80]:
store = pd.HDFStore(r"C:\Users\ngocna3\Documents\My Received Files\Work\CD_IRRBB\FD_2022\Data\FD_BALANCE_310822.h5") # cbal
store.open()
#input xlkt
xlkt_acc_1 = pd.read_excel(data_folder+ r"\Gdkt_upto_72022 - Account.xlsx").ACCTNO.unique()
xlkt_acc_3=store.select("data",columns=["ACCTNO","CIFNO","TYPE"]) # account xlkt
CD_and_xlkt_acc_3=xlkt_acc_3[(xlkt_acc_3.CIFNO.isin([290445,949938,290445,132738,1599267])) | # khách hàng affiliate
                             (xlkt_acc_3.TYPE.str.startswith("CD"))].ACCTNO.unique() # account CD
xlkt_acc=xlkt_acc_1.tolist()+CD_and_xlkt_acc_3.tolist() 

In [ ]:
# xlkt
# xlkt_acc_1=pd.read_excel(data_folder+r"\xlkt_31082020.xlsx",sheet_name="FCRM",usecols=[2]).ACCOUNT_NO.unique() # account FCRM rút vốn linh hoạt
# xlkt_acc_2=pd.read_csv(data_folder+r"\xlkt.csv").ACCTNO.unique() # account FCRM rút vốn linh hoạt
# Xử lý account
# store = pd.HDFStore(path.join(data_folder,"FD_BALANCE.h5")) # cbal
# store.open()
# xlkt_acc_3=store.select("data",columns=["ACCTNO","CIFNO","TYPE"]) # account xlkt
# CD_and_xlkt_acc_3=xlkt_acc_3[(xlkt_acc_3.CIFNO.isin([290445,949938,290445,132738,1599267])) | # khách hàng affiliate
#                              (xlkt_acc_3.TYPE.str.startswith("CD"))].ACCTNO.unique() # account CD
# xlkt_acc=xlkt_acc_1.tolist()+xlkt_acc_2.tolist()+CD_and_xlkt_acc_3.tolist() 

In [81]:
# lấy data từ file H5
# data sau khi lấy xong sẽ có ngày cuối cùng trùng với ngày end lựa chọn của mô hình (chỉ cần quan tâm file H5 có ngày end ko nhỏ hơn ngày end của mô hình là được)
#Import file dữ liệu
# store = pd.HDFStore(r"C:\Users\ngocna3\Documents\irrbb\FD_2022\Data\FD_BALANCE_31082022.h5") # cbal
# store.open()
data=store.select("data",auto_close=True)
data.loc[data.CHECK_TO_DATE == pd.Timestamp('2022-01-02'), 'CHECK_TO_DATE'] = pd.Timestamp('2022-01-03') 
data.loc[data.CHECK_TO_DATE == pd.Timestamp('2022-03-13'), 'CHECK_TO_DATE'] = pd.Timestamp('2022-03-14') 
data.loc[data.CHECK_TO_DATE == pd.Timestamp('2022-03-25'), 'CHECK_TO_DATE'] = pd.Timestamp('2022-03-26') 
data=data[(data.CHECK_FROM_DATE<=end_date) & (data.ACCTNO.isin(xlkt_acc)==False) # acc xlkt
        & (data.TYPE!="FCRM106") #ung von
        & (data.TYPE.str.contains("PW")==False) # rut von linh hoat
        & (data.CURRENT_BALANCE > 0 )].reset_index(drop=True).sort_values("CHECK_FROM_DATE")
data=handling_data(data,bank,sfs,list_date,cal_cbal=False)
data["bank"]=data["bank"].map(lambda x: "RC" if (x in list("RC")) else ("LSB" if (x in list("LSB")) else "F_NF"))
data.loc[data.CHECK_TO_DATE>end_date,"CHECK_TO_DATE"]=end_date 

In [61]:
'''
code cũ: ppl tính TW
'''
# thống kê account
def return_df_tw(date,table=data,danhmuc=[None,None],method_mat="All_0"):
    # ví dụ danhmuc=["VND","RC"]
    tw=int(method_mat.split("_")[-1])
    method=method_mat.split("_")[0]
    c_all= (True if ("All" in method) else False)
    c_mat_after = ((table.MAT_DATE>(date+pd.Timedelta(str(tw)+"D"))) if ("After" in method) else False) # account có mat-date lớn hơn posted + tw
    c_mat_equal = ((table.MAT_DATE==(date+pd.Timedelta(str(tw)+"D"))) if ("Equal" in method) else False) # account có mat-date bằng posted + tw
    c_mat_before = ((table.MAT_DATE<(date+pd.Timedelta(str(tw)+"D")))  if ("Before" in method) else False) # account có mat-date nhỏ hơn posted + tw
    condi_mat= (((c_all | c_mat_after) | c_mat_equal) | c_mat_before)
    condi_cur= (True if (danhmuc[0]==None) else (table.CURTYP==danhmuc[0]))
    condi_bank= (True if (danhmuc[1]==None) else (table.bank==danhmuc[1]))
    return table[ (table.CHECK_FROM_DATE<=date) & (table.CHECK_TO_DATE>=date) & condi_cur & condi_bank & condi_mat].copy()

def calculate_sum_balance(date,t,table=data,key="ALL",func=np.nansum):        
    df = table[["CURTYP","bank","CURRENT_BALANCE"]].groupby(["CURTYP","bank"]).agg(func)
    if df.columns.nlevels==2:
        df=df.melt(ignore_index=False,col_level=1,var_name=["func"],value_name=date).set_index("func",append=True)
    else:
        df.columns=[date]
    df[["KIND","TW"]]=[key,t]
    df.set_index(["KIND","TW"],append=True,inplace=True)
    return df.T

def Nan_to_tw(table,level=3):
    table.fillna(0,inplace=True)
    for i in table.columns:
        if int(i[level])>0:table.loc[table.index[-int(i[level]):],i]=np.nan

Result=pd.DataFrame()
list_tw=[30,90,180,365,730,1095, 1460, 1825, 2190]
for date in list_date:
    Result_day=pd.DataFrame()
    df_0=return_df_tw(date,method_mat="All_{}".format(0)) # danh mục FD có posted = date (ký hiệu DF0)
    for tw in list_tw:
        print(date.date(),tw,end="\r")
        all_after=return_df_tw(date,table=df_0,method_mat="After_{}".format(tw)) # danh mục con của DF0 (posted=date) có acc có mat-date sau ngày date+tw
        all_after_sum=calculate_sum_balance(date,tw,table=all_after,key="ALL") # tính tổng giá trị danh mục all_after
        date_tw=date+pd.Timedelta("{}D".format(tw)) # date+tw
        all_tw=return_df_tw(date_tw,table=data,method_mat="All_{}".format(tw)) # danh mục có posted=date_tw
        pre_after_sum=calculate_sum_balance(date,tw,table=all_after[~all_after.ACCTNO.isin(all_tw.ACCTNO)],
                                        key="PRE") # tính tổng danh mục của những account trong all_after nhưng không xuất hiện trong danh mục all ngày date+tw (tức số dư đã về 0 trước đó)
        Result_day=pd.concat([Result_day,all_after_sum,pre_after_sum],axis=1)
        clear_output()
    Result=Result.append(Result_day)
Nan_to_tw(Result)
BWex=modify_columns_name(Result.iloc[:,Result.columns.get_level_values(1)=="RC"],1,"BWex",axis=1)\
.add(modify_columns_name(Result.iloc[:,Result.columns.get_level_values(1)=="LSB"],1,"BWex",axis=1),fill_value=0)
Result=pd.concat([Result,BWex],axis=1).sort_index(axis=1)
# export
Result.to_pickle(output_folder+r"\Result_310822.pickle")

NameError: name 'data' is not defined

In [87]:
'''
code mới, tính theo bucket - phương pháp luận mới
'''
 
####
def return_df_tw2(date,table=data,danhmuc=[None,None],method_mat="ALL_0_1"):
    # Tw1, Tw2 là hai giá trị chặn trong Bucket muốn xét
    # Method = ALL => xác định balance có MAT > TW2, Method = wBetween => Xác định withdrawal between Tw1 và TW2
    tw1=int(method_mat.split("_")[1])
    tw2=int(method_mat.split("_")[-1])
    method = method_mat.split("_")[0]
    c_mat_1 = ((table.MAT_DATE>(date+pd.Timedelta(str(tw2)+"D")))\
               if ("ALL" in method) else False)  
    #các điều kiện xác định account rút trước hạn (mat>tw2 & rút trước hạn trong bucket đang xét)
    c_wit_cond1 = ((table.MAT_DATE>(date+pd.Timedelta(str(tw2)+"D"))) if ("wbetween" in method) else False)
    c_wit_1 = ((table.CHECK_TO_DATE>=(date+pd.Timedelta(str(tw1)+"D")) if (tw1 == 0) else table.CHECK_TO_DATE>=(date+pd.Timedelta(str(tw1)+"D")))\
               if ("wbetween" in method) else False)
    c_wit_2 = ((((table.CHECK_TO_DATE )<(date+pd.Timedelta(str(tw2)+"D"))))  if ("wbetween" in method) else False) 
    #c_wit_2 = (((table.CHECK_TO_DATE + pd.Timedelta(str(1)+"D"))<(date+pd.Timedelta(str(tw2)+"D"))) if ("wbetween" in method) else False)  
    c_wit_between = (c_wit_cond1 & c_wit_1 &  c_wit_2)
    
    return table [(table.CHECK_FROM_DATE<=date) & (table.CHECK_TO_DATE>=date) & ( c_mat_1 | c_wit_between) ].copy()


def calculate_sum_balance(date,t,table=data,key="ALL",func=np.nansum):        
    df = table[["CURTYP","bank","CURRENT_BALANCE"]].groupby(["CURTYP","bank"]).agg(func)
    if df.columns.nlevels==2:
        df=df.melt(ignore_index=False,col_level=1,var_name=["func"],value_name=date).set_index("func",append=True)
    else:
        df.columns=[date]
    df[["KIND","Bucket"]]=[key,t]
    df.set_index(["KIND","Bucket"],append=True,inplace=True)
    return df.T

def Nan_to_tw(table,level=3):
    table.fillna(0,inplace=True)
    for i in table.columns:
        if int(i[level].split('-')[1].split('D')[0])>0: table.loc[table.index[-int(i[level].split('-')[1].split('D')[0]):],i]=np.nan

Result=pd.DataFrame()
list_tw1=[0,30,90,180,365,730,1095,1460,1825]
list_tw2=[30,90,180,365,730,1095,1460,1825,2190]
# list_tw1=[0]
# list_tw2=[30]
for date in list_date[:2]:
    Result_day=pd.DataFrame()
    for i in range(9):
        print(date.date(),list_tw1[i],list_tw2[i],end="\r")
        all_mat_bucket=return_df_tw2(date,table=data,method_mat="ALL_{}_{}".format(list_tw1[i],list_tw2[i])) # trả về bảng có mat > date + tw2
        all_mat_bucket_sum=calculate_sum_balance(date,t = '{}-{}'.format(list_tw1[i],list_tw2[i]),table=all_mat_bucket,key="ALL") # tính tổng giá trị all_mat_bucket
        Result_day=pd.concat([Result_day,all_mat_bucket_sum],axis=1)
        date_tw1=date+pd.Timedelta("{}D".format(list_tw1[i])) # date+tw1
        date_tw2=date+pd.Timedelta("{}D".format(list_tw2[i])) # date+tw2
        all_tw1=return_df_tw2(date_tw1,table=data,method_mat="ALL_0_0") # danh mục tồn tại tại ngày date + tw1
        all_tw2=return_df_tw2(date_tw2,table=data,method_mat="ALL_0_0") # danh mục tồn tại tại ngày date + tw2
        all_wit_bucket = all_mat_bucket[all_mat_bucket.ACCTNO.isin(all_tw1.ACCTNO)][~all_mat_bucket.ACCTNO.isin(all_tw2.ACCTNO)]
        all_wit_bucket_sum=calculate_sum_balance(date,t ='{}-{}'.format(list_tw1[i],list_tw2[i]),table=all_wit_bucket,key="PRE") #tính bảng có account RTHall_mat_bucket        
        Result_day=pd.concat([Result_day,all_wit_bucket_sum],axis=1)
        clear_output()
    Result =Result.append(Result_day)
    

Result=Result.sort_index(axis=1, level = [0,1,2], sort_remaining = False)
BWex=modify_columns_name(Result.iloc[:,Result.columns.get_level_values(1)=="RC"],1,"BWex",axis=1).add(modify_columns_name(Result.iloc[:,Result.columns.get_level_values(1)=="LSB"],1,"BWex",axis=1),fill_value=0)
Result=pd.concat([Result,BWex],axis=1).sort_index(axis=1, level = [0], sort_remaining = False)
Nan_to_tw(table=Result,level=3)
#Result.to_pickle(r"D:\IRRBB 2022\output_T10_2022\Result_31102022.pickle") 
Result


In [ ]:
# hàm NaN của code mới
def Nan_to_tw(table,level=3):
    table.fillna(0,inplace=True)
    for i in table.columns:
        if int(i[level].split('-')[1].split('D')[0])>0: table.loc[table.index[-int(i[level].split('-')[1].split('D')[0]):],i]=np.nan

In [89]:
Result.to_pickle(output_folder+r"Result_31102021_v2.pickle")

In [102]:
Result.to_csv("test_send_Huong.csv")

# Xử lý file output

In [90]:
import pandas as pd
from os import path,getcwd
import numpy as np
from IPython.display import clear_output

In [93]:
output_folder

'C:\\Users\\ngocna3\\Documents\\My Received Files\\Work\\CD_IRRBB\\FD_2022\\Output'

In [94]:
# path
# path
code_folder=getcwd()
output_folder=path.join(code_folder,'Output')
data_folder =path.join(code_folder,'Data')

#Import file dữ liệu
#Result=pd.read_pickle(r"C:\Users\ngocna3\Documents\My Received Files\Work\CD_IRRBB\FD_2022\OutputResult_31102021_v2.pickle")
Result = pd.read_csv(r")
# Khai báo seasonal
Solar=[pd.Timestamp(year=y,month=1,day=1)for y in range(2016,2025)]
Lunar=[pd.to_datetime(i,dayfirst=True) for i in ['8/2/2016', '28/1/2017', '16/2/2018', '5/2/2019', '25/1/2020','12/02/2021','01/02/2022','22/01/2023',"10/02/2024"]]
ss_day_t0=[x for y in [pd.date_range(Solar[i]-pd.Timedelta("45D"),Lunar[i]-pd.Timedelta("30D")) for i in range(len(Solar))] for x in y]

In [97]:
''' old code '''
def WD_PRE_ratio_func(i,pre_t_tw_t,all_0_tw_t):
    i_=list(i).copy()
    i__=list(i).copy()
    i_[2]="ALL"
    i__[2]="WD_PRE_RATIO"
    wdr_pre=(pre_t_tw_t[i]/all_0_tw_t[tuple(i_)]).fillna(0).copy()
    #wdr_pre.iloc[-i[3]:]=np.nan
    return pd.DataFrame(wdr_pre,columns=pd.MultiIndex.from_tuples([tuple(i__)],names=pre_t_tw_t.columns.names))
def Nan_to_tw(table,level=3):
    table.fillna(0,inplace=True)
    for i in table.columns:
        if int(i[level])>0:table.loc[table.index[-int(i[level]):],i]=np.nan

In [ ]:
''' new code '''
# verse 2
def W_ratio(i,pre_t_tw_t,all_0_tw_t):
    i_=list(i).copy()
    i__=list(i).copy()
    i_[2]="ALL"
    i__[2]="WD_RATIO"
    wdr_pre=(pre_t_tw_t[i]/all_0_tw_t[tuple(i_)]).fillna(0).copy()
    return pd.DataFrame(wdr_pre,columns=pd.MultiIndex.from_tuples([tuple(i__)],names=pre_t_tw_t.columns.names))

def Nan_to_tw(table,level=3):
    table.fillna(0,inplace=True)
    for i in table.columns:
        if int(i[level].split('-')[1].split('D')[0])>0: table.loc[table.index[-int(i[level].split('-')[1].split('D')[0]):],i]=np.nan

In [98]:
# tính WD ratio cho rút trước hạn
all_0_tw_t=Result.iloc[:, (Result.columns.get_level_values("KIND")=='ALL') ].copy()
pre_t_tw_t=Result.iloc[:, (Result.columns.get_level_values("KIND")=='PRE')].copy()
wd_ratio=pd.concat([ WD_PRE_ratio_func(i,pre_t_tw_t,all_0_tw_t) for i in pre_t_tw_t.columns],axis=1)
#wd_ratio.to_pickle(output_folder+r"\wd_ratio_310822_v2.pickle")

In [177]:
#new
wd_ratio = pd.read_excel(r"C:\Users\ngocna3\Documents\My Received Files\Ratio_ver2(1).xlsx", header = [0,1,2,3], index_col = [0])

In [212]:
#ngocna3 edits
def Nan_to_tw(table):
    table.fillna(0,inplace=True)
    for i in table.columns:
        if int(i[3].split('-')[1].split('D')[0])>0:table.loc[table.index[-int(i[3].split('-')[1].split('D')[0]):],i]=np.nan

In [218]:
wd_ratio.tail(180)

CURTYP         OTHER                                                         \
bank              RC                                 BWex                     
KIND        WD_RATIO                             WD_RATIO                     
Bucket         0-30D    30-90D 90-180D 180-270D     0-30D    30-90D 90-180D   
2022-03-05  0.107416  0.000301     NaN      NaN  0.107416  0.000301     NaN   
2022-03-06  0.109367  0.000312     NaN      NaN  0.109367  0.000312     NaN   
2022-03-07  0.094899  0.000294     NaN      NaN  0.094899  0.000294     NaN   
2022-03-08  0.062384  0.000271     NaN      NaN  0.062384  0.000271     NaN   
2022-03-09  0.088719  0.000271     NaN      NaN  0.088719  0.000271     NaN   
...              ...       ...     ...      ...       ...       ...     ...   
2022-08-27       NaN       NaN     NaN      NaN       NaN       NaN     NaN   
2022-08-28       NaN       NaN     NaN      NaN       NaN       NaN     NaN   
2022-08-29       NaN       NaN     NaN      NaN       NaN       NaN     NaN   
2022-08-30       NaN       NaN     NaN      NaN       NaN       NaN     NaN   
2022-08-31       NaN       NaN     NaN      NaN       NaN       NaN     NaN   

CURTYP                   USD         ...       VND                             \
bank                     LSB         ...      BWex                              
KIND                WD_RATIO         ...  WD_RATIO                              
Bucket     180-270D    0-30D 30-90D  ...     0-30D    30-90D 90-180D 180-270D   
2022-03-05      NaN      0.0    0.0  ...  0.027316  0.021972     NaN      NaN   
2022-03-06      NaN      0.0    0.0  ...  0.026497  0.021775     NaN      NaN   
2022-03-07      NaN      0.0    0.0  ...  0.030882  0.022785     NaN      NaN   
2022-03-08      NaN      0.0    0.0  ...  0.030300  0.022715     NaN      NaN   
2022-03-09      NaN      0.0    0.0  ...  0.030058  0.021746     NaN      NaN   
...             ...      ...    ...  ...       ...       ...     ...      ...   
2022-08-27      NaN      NaN    NaN  ...       NaN       NaN     NaN      NaN   
2022-08-28      NaN      NaN    NaN  ...       NaN       NaN     NaN      NaN   
2022-08-29      NaN      NaN    NaN  ...       NaN       NaN     NaN      NaN   
2022-08-30      NaN      NaN    NaN  ...       NaN       NaN     NaN      NaN   
2022-08-31      NaN      NaN    NaN  ...       NaN       NaN     NaN      NaN   

CURTYP                                                                   
bank                                                                     
KIND                                                                     
Bucket     270-365D 365-730D 730-1095D 1095-1460D 1460-1825D 1825-2190D  
2022-03-05      NaN      NaN       NaN        NaN        NaN        NaN  
2022-03-06      NaN      NaN       NaN        NaN        NaN        NaN  
2022-03-07      NaN      NaN       NaN        NaN        NaN        NaN  
2022-03-08      NaN      NaN       NaN        NaN        NaN        NaN  
2022-03-09      NaN      NaN       NaN        NaN        NaN        NaN  
...             ...      ...       ...        ...        ...        ...  
2022-08-27      NaN      NaN       NaN        NaN        NaN        NaN  
2022-08-28      NaN      NaN       NaN        NaN        NaN        NaN  
2022-08-29      NaN      NaN       NaN        NaN        NaN        NaN  
2022-08-30      NaN      NaN       NaN        NaN        NaN        NaN  
2022-08-31      NaN      NaN       NaN        NaN        NaN        NaN  

[180 rows x 58 columns]

## Phương pháp cắt percentile

### Tính percentile

In [297]:
wd_ratio = pd.read_csv(r")

SyntaxError: EOL while scanning string literal (<ipython-input-297-64fcef006cbd>, line 1)

In [ ]:
wd_ratio= pd.read_excel(r" ",header =[0,1,2,3], index_col =[0])

In [222]:
# tập train truoc 30 ngay
wd_ratio_train=wd_ratio.iloc[:-30].copy()
Nan_to_tw(wd_ratio_train)
#wd_ratio_train = wd_ratio_train.drop(2190, axis = 1, level = 3) #khi fill = np.nan thi cot 2190 nan het

def calculate_percentile(good_or_bad="bad",wd_ratio_train=wd_ratio_train):
    per,st_rank= (90,-10) if good_or_bad == "bad" else (10,9)
    
    # tính bộ tỷ lệ trong các điều kiện persentile 90%
    normal=wd_ratio_train[ (~wd_ratio_train.index.isin(ss_day_t0))].apply(lambda x:np.nanpercentile(x.iloc[-365-x.isna().sum():],per))
    seasonal=wd_ratio_train[wd_ratio_train.index.isin(ss_day_t0)].apply(lambda x:np.nanpercentile(x,per))
    stress=wd_ratio_train.apply(lambda x:np.sort(x[~np.isnan(x)])[st_rank])

    wd_ratio_percentile=pd.concat([normal,seasonal,stress],axis=1).rename(columns={0:"Normal",1:"Seasonal",2:"Stress"}).reset_index()
    wd_ratio_percentile=wd_ratio_percentile.melt(id_vars=['CURTYP','bank',"Bucket","KIND"],value_vars=['Normal','Seasonal','Stress'],var_name="Time",value_name="Ratio")\
        .pivot_table(values="Ratio",columns=['KIND',"Bucket"],index=['Time','CURTYP','bank'])
    # wd_ratio_percentile=wd_ratio_percentile.applymap(lambda x:'{:.1%}'.format(x))
    wd_ratio_percentile.reset_index(inplace=True)
    wd_ratio_percentile=wd_ratio_percentile[(wd_ratio_percentile.CURTYP!="OTHER") 
                                            & ~(wd_ratio_percentile.bank.isin(["F_NF"])) 
                                            & ~((wd_ratio_percentile.bank.isin(["BWex","LSB"])) & (wd_ratio_percentile.CURTYP=="USD"))]\
    .sort_values(["Time","CURTYP","bank"],ascending=[True,False,True])
    return wd_ratio_percentile

In [223]:
writer= pd.ExcelWriter(output_folder+r"\wd_ratio_percentile_310822_new_method.xlsx",engine="openpyxl")#mode="+a")
calculate_percentile("good").to_excel(writer,sheet_name="good")
calculate_percentile("bad").to_excel(writer,sheet_name="bad")
writer.save()

#calculate_percentile("bad").to_pickle(output_folder+r"\wd_ratio_percentile_310822.pickle")

In [ ]:
calculate_percentile("bad").to_pickle(output_folder+r"\wd_ratio_percentile_310822.pickle")

### Backtest percentile

In [225]:
import pandas as pd
from os import path,getcwd
import numpy as np

# path
code_folder=getcwd()
output_folder=path.join(code_folder,'Output')
data_folder =path.join(code_folder,'Data')

#Import file dữ liệu
Result=pd.read_excel(r"C:\Users\ngocna3\Documents\My Received Files\Result_ver2.xlsx", header = [0,1,2,3], index_col = [0])
Nan_to_tw(Result)
wd_ratio_percentile=calculate_percentile("bad")

# Khai báo seasonal
Solar=[pd.Timestamp(year=y,month=1,day=1)for y in range(2016,2025)]
Lunar=[pd.to_datetime(i,dayfirst=True) for i in ['8/2/2016', '28/1/2017', '16/2/2018', '5/2/2019', '25/1/2020','12/02/2021','01/02/2022','22/01/2023',"10/02/2024"]]
ss_day_t0=[x for y in [pd.date_range(Solar[i]-pd.Timedelta("45D"),Lunar[i]-pd.Timedelta("30D")) for i in range(len(Solar))] for x in y]

#Import file dữ liệu
day_bt_list=[30,90,180,365,730,1095]

In [229]:
''' code cũ - dùng cho tw '''
def cut_off_obs(df,day_bt,condi,num_test_obs): # lọc quan sát theo điều kiện
    df2=df.iloc[:-day_bt].copy()
    if condi.lower()=="normal":
        df2=df2[~df2.index.isin(ss_day_t0)].iloc[-365-int(num_test_obs):]
    elif condi.lower()=="seasonal":
        df2=df2[df2.index.isin(ss_day_t0)]
    else: #stress
        df2=df2
    return df2

def calculate_backtest(report,wd_percentile_raw,b,c,condi,bt,num_test_obs):
    df=report.iloc[:,(report.columns.get_level_values("CURTYP")==c) & (report.columns.get_level_values("bank")==b) 
                     & (report.columns.get_level_values("Bucket")==bt)].copy().sort_index(level=["KIND"],axis=1)
        
    df.columns=["CB0","CB_T"]
    wd_ratio=wd_percentile_raw.loc[(wd_percentile_raw.Time==condi.capitalize())
                     &(wd_percentile_raw.CURTYP==c)
                     &(wd_percentile_raw.bank==b),wd_percentile_raw.columns.get_level_values("Bucket")==bt].values[0][0]
    df["WD_Estimate"]=wd_ratio*df["CB0"]
    df["Violation"]=(df["CB_T"]>df["WD_Estimate"]).astype(int)
    df["Balance_violation"]=df["Violation"]*(df["CB_T"]-df["WD_Estimate"])
    df["%Balance_violation"]=df["Balance_violation"]/df["CB0"]
    func={"Violation":np.nansum,"Balance_violation":np.nanmean,"%Balance_violation":[np.nanmax,np.nanmean]}
    df[["Bank","CCY","Condition","Bucket"]]=[b,c,condi,bt]
    df["CB_T_in_SS"]=df["CB_T"]*df.index.map(lambda x: np.nan if x not in ss_day_t0 else 1)
    df_cut_off=cut_off_obs(df=df,day_bt=bt,condi=condi,num_test_obs=num_test_obs)
    df_month=df_cut_off.reset_index().groupby(pd.Grouper(key='index',freq="M")).agg(func).reset_index()
    df_month.columns=df_month.columns.map(lambda x: "_".join(x))
    df_month[["Bank","CCY","Condition","Bucket"]]=[b,c,condi,bt]
    return df.iloc[:-bt].reset_index(),df_month

In [230]:
''' code cũ '''
# BACKTEST
num_test_obs=30
backtest_result_day=backtest_result_month=pd.DataFrame()
for b,c in  [("RC","VND"),("RC","USD"),("LSB","VND"),("BWex","VND")]:
    for bt in day_bt_list:
        for condition in ["Stress","Seasonal","Normal"]:#
            df,df_month=calculate_backtest(report=Result,wd_percentile_raw=wd_ratio_percentile,b=b,c=c,condi=condition,bt=bt,num_test_obs=num_test_obs)
            backtest_result_day=backtest_result_day.append(df)
            backtest_result_month=backtest_result_month.append(df_month)

backtest_result_day_CB_T=backtest_result_day.drop_duplicates(["index","Bank","CCY","Bucket"]).copy().drop(["WD_Estimate",'Violation','Balance_violation','%Balance_violation',"CB0"],axis=1)
backtest_result_day_CB_T1=backtest_result_day_CB_T.rename(columns={"CB_T":"WD_Estimate"}).drop("CB_T_in_SS",axis=1)
backtest_result_day_CB_T1["Condition"]="Lượng rút trước hạn"
backtest_result_day_CB_T2=backtest_result_day_CB_T.rename(columns={"CB_T_in_SS":"WD_Estimate"}).drop("CB_T",axis=1)
backtest_result_day_CB_T2["Condition"]="Lượng rút trước hạn mùa vụ"
backtest_result_day=backtest_result_day.append(backtest_result_day_CB_T1).append(backtest_result_day_CB_T2)
backtest_result_day.WD_Estimate.replace(0,np.nan,inplace=True)
backtest_result_day=backtest_result_day[backtest_result_day.WD_Estimate.notna()]
backtest_result_day=backtest_result_day.drop(backtest_result_day.columns[backtest_result_day.isna().any(0)],axis=1)            

with pd.ExcelWriter(output_folder+r"\Plot_Backtest_FD_IRRBB_310822_new_method.xlsx",engine='openpyxl') as writer:
    backtest_result_day.to_excel(writer,sheet_name="BACKTEST DAY",index=False)
    backtest_result_month.to_excel(writer,sheet_name="BACKTEST MONTH",index=False)

ValueError: Length mismatch: Expected axis has 0 elements, new values have 2 elements

In [ ]:
''' code mới - dùng cho bucket '''
num_test_obs=92

def cut_off_obs(df,day_bt,condi,num_test_obs): # lọc quan sát theo điều kiện
    df2=df.iloc[:-(day_bt)].copy()
    if condi.lower()=="normal":
        df2=df2[~df2.index.isin(ss_day_t0)].iloc[-365-int(num_test_obs):]
    elif condi.lower()=="seasonal":
        df2=df2[df2.index.isin(ss_day_t0)]
    else: #stress
        df2=df2
    return df2

def calculate_backtest(report,wd_percentile_raw,b,c,condi,bt,num_test_obs):
    df=report.iloc[:,(report.columns.get_level_values("CURTYP")==c) & (report.columns.get_level_values("bank")==b) 
                     & (report.columns.get_level_values("Bucket")==bt)].copy().sort_index(level=["KIND"],axis=1)
    df.columns=["CB0","CB_T"]
    wd_ratio=wd_percentile_raw.loc[(wd_percentile_raw.Time==condi.capitalize())
                     &(wd_percentile_raw.CURTYP==c)
                     &(wd_percentile_raw.bank==b),wd_percentile_raw.columns.get_level_values("Bucket")==bt].values[0][0]
    df["WD_Estimate"]=wd_ratio*df["CB0"]
    df["Violation"]=(df["CB_T"]>df["WD_Estimate"]).astype(int)
    df["Balance_violation"]=df["Violation"]*(df["CB_T"]-df["WD_Estimate"])
    df["%Balance_violation"]=df["Balance_violation"]/df["CB0"]
    func={"Violation":np.nansum,"Balance_violation":np.nanmean,"%Balance_violation":[np.nanmax,np.nanmean]}
    df[["Bank","CCY","Condition","Bucket"]]=[b,c,condi,bt]
    df["CB_T_in_SS"]=df["CB_T"]*df.index.map(lambda x: np.nan if x not in ss_day_t0 else 1)
    df_cut_off=cut_off_obs(df=df,day_bt=int(bt.split('-')[1].split('D')[0]),condi=condi,num_test_obs=num_test_obs)
    df_month=df_cut_off.reset_index().groupby(pd.Grouper(key='index',freq="M")).agg(func).reset_index()
    df_month.columns=df_month.columns.map(lambda x: "_".join(x))
    df_month[["Bank","CCY","Condition","Bucket"]]=[b,c,condi,bt]
    return df.iloc[:-(int(bt.split('-')[1].split('D')[0]))].reset_index(),df_month
    #return df.reset_index(),df_month
   

In [ ]:
# BACKTEST
num_test_obs=30
backtest_result_day=pd.DataFrame()
backtest_result_month=pd.DataFrame()
for b,c in  [("RC","VND"),("RC","USD"),("LSB","VND"),("BWex","VND")]:
    for bt in ['0-30','30-90','90-180','180-365','365-730','730-1095']:
        for condition in ["Stress","Seasonal","Normal"]:#
            df,df_month=calculate_backtest(report=Result,wd_percentile_raw=wd_ratio_percentile,b=b,c=c,condi=condition,bt=bt,num_test_obs=num_test_obs)
            backtest_result_day=backtest_result_day.append(df)
            backtest_result_month=backtest_result_month.append(df_month)

backtest_result_day_CB_T=backtest_result_day.drop_duplicates(["index","Bank","CCY","Bucket"]).copy().drop(["WD_Estimate",'Violation','Balance_violation','%Balance_violation',"CB0"],axis=1)
backtest_result_day_CB_T1=backtest_result_day_CB_T.rename(columns={"CB_T":"WD_Estimate"}).drop("CB_T_in_SS",axis=1)
backtest_result_day_CB_T1["Condition"]="Lượng rút trước hạn"
backtest_result_day_CB_T2=backtest_result_day_CB_T.rename(columns={"CB_T_in_SS":"WD_Estimate"}).drop("CB_T",axis=1)
backtest_result_day_CB_T2["Condition"]="Lượng rút trước hạn mùa vụ"
backtest_result_day=backtest_result_day.append(backtest_result_day_CB_T1).append(backtest_result_day_CB_T2)
backtest_result_day.WD_Estimate.replace(0,np.nan,inplace=True)
backtest_result_day=backtest_result_day[backtest_result_day.WD_Estimate.notna()]
backtest_result_day=backtest_result_day.drop(backtest_result_day.columns[backtest_result_day.isna().any(0)],axis=1)            

with pd.ExcelWriter(r"D:\IRRBB 2022\output_T10_2022\Backtest_FD_IRRBB_10_2022_thử lại lần 7.xlsx",engine='openpyxl') as writer:
    backtest_result_day.to_excel(writer,sheet_name="BACKTEST DAY",index=False)
    backtest_result_month.to_excel(writer,sheet_name="BACKTEST MONTH",index=False)

## Hồi quy

In [231]:
import pandas as pd
from os import path,getcwd
import os
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import datetime
import matplotlib.dates as mdates

from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from scipy.ndimage.filters import gaussian_filter1d
import warnings
# import ast
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline

warnings.filterwarnings("ignore")

### Khai báo biến

In [232]:
# path
code_folder=getcwd()
output_folder=path.join(code_folder,'Output')
data_folder =path.join(code_folder,'Data')

# Khai báo seasonal
Solar=[pd.Timestamp(year=y,month=1,day=1)for y in range(2016,2025)]
Lunar=[pd.to_datetime(i,dayfirst=True) for i in ['8/2/2016', '28/1/2017', '16/2/2018', '5/2/2019', '25/1/2020','12/02/2021','01/02/2022','22/01/2023',"10/02/2024"]]
ss_day_t0=[x for y in [pd.date_range(Solar[i]-pd.Timedelta("45D"),Lunar[i]-pd.Timedelta("30D")) for i in range(len(Solar))] for x in y]
st_msb=pd.date_range("2016-07-01","2016-10-12")

# kết quả percentile ==> tạo kết quả mix
#per=pd.read_pickle(output_folder+r"\Outputket_qua_ver2_Huong_send_241022.pickle")

### Xử lý dữ liệu

#### Biến ngày

In [233]:
start_day_reg=pd.Timestamp("2018-07-02") # Ngày đầu tiên chạy hồi quy
end_day_reg=pd.Timestamp("2022-08-31")
end_day_reg_train=pd.Timestamp("2022-07-31")
#số ngày giai đoạn test
test_size=int((end_day_reg-end_day_reg_train)/pd.Timedelta("1D"))

#covid 2020
covid_20 = pd.date_range(pd.Timestamp("2020-02-14"),pd.Timestamp("2020-07-01")).to_list()
covid_21 = pd.date_range(pd.Timestamp("2021-07-09"),pd.Timestamp("2021-10-11")).to_list()
covid = covid_20 + covid_21

# Khai báo seasonal
Solar=[pd.Timestamp(year=y,month=1,day=1)for y in range(2016,2025)]
Lunar=[pd.to_datetime(i,dayfirst=True) for i in ['8/2/2016', '28/1/2017', '16/2/2018', '5/2/2019', '25/1/2020','12/02/2021','01/02/2022','22/01/2023',"10/02/2024"]]
ss_day_t0=[x for y in [pd.date_range(Solar[i]-pd.Timedelta("45D"),Lunar[i]-pd.Timedelta("30D")) for i in range(len(Solar))] for x in y]


#### Biến phụ thuộc

In [234]:
Withdraw_Ratio = wd_ratio.copy()

In [235]:
Withdraw_Ratio

CURTYP         OTHER                                                   \
bank              RC                                   BWex             
KIND        WD_RATIO                               WD_RATIO             
Bucket         0-30D    30-90D   90-180D 180-270D     0-30D    30-90D   
2015-10-31  0.008545  0.017176  0.184745      0.0  0.008545  0.017176   
2015-11-01  0.008545  0.017193  0.184745      0.0  0.008545  0.017193   
2015-11-02  0.008870  0.017034  0.184745      0.0  0.008870  0.017034   
2015-11-03  0.008958  0.017372  0.178573      0.0  0.008958  0.017372   
2015-11-04  0.007537  0.017172  0.178573      0.0  0.007537  0.017172   
...              ...       ...       ...      ...       ...       ...   
2022-08-27       NaN       NaN       NaN      NaN       NaN       NaN   
2022-08-28       NaN       NaN       NaN      NaN       NaN       NaN   
2022-08-29       NaN       NaN       NaN      NaN       NaN       NaN   
2022-08-30       NaN       NaN       NaN      NaN       NaN       NaN   
2022-08-31       NaN       NaN       NaN      NaN       NaN       NaN   

CURTYP                             USD         ...       VND            \
bank                               LSB         ...      BWex             
KIND                          WD_RATIO         ...  WD_RATIO             
Bucket       90-180D 180-270D    0-30D 30-90D  ...     0-30D    30-90D   
2015-10-31  0.184745      0.0      0.0    1.0  ...  0.017849  0.012629   
2015-11-01  0.184745      0.0      0.0    1.0  ...  0.018198  0.012420   
2015-11-02  0.184745      0.0      0.0    1.0  ...  0.018068  0.011886   
2015-11-03  0.178573      0.0      0.0    1.0  ...  0.018516  0.011718   
2015-11-04  0.178573      0.0      0.0    1.0  ...  0.018152  0.011689   
...              ...      ...      ...    ...  ...       ...       ...   
2022-08-27       NaN      NaN      NaN    NaN  ...       NaN       NaN   
2022-08-28       NaN      NaN      NaN    NaN  ...       NaN       NaN   
2022-08-29       NaN      NaN      NaN    NaN  ...       NaN       NaN   
2022-08-30       NaN      NaN      NaN    NaN  ...       NaN       NaN   
2022-08-31       NaN      NaN      NaN    NaN  ...       NaN       NaN   

CURTYP                                                                   \
bank                                                                      
KIND                                                                      
Bucket       90-180D  180-270D  270-365D  365-730D 730-1095D 1095-1460D   
2015-10-31  0.027391  0.027198  0.040842  0.007360  0.000594        0.0   
2015-11-01  0.027483  0.027044  0.043161  0.007360  0.000594        0.0   
2015-11-02  0.028124  0.027009  0.041134  0.007330  0.000557        0.0   
2015-11-03  0.028000  0.026968  0.039527  0.007354  0.000000        0.0   
2015-11-04  0.027738  0.027196  0.038515  0.007364  0.000000        0.0   
...              ...       ...       ...       ...       ...        ...   
2022-08-27       NaN       NaN       NaN       NaN       NaN        NaN   
2022-08-28       NaN       NaN       NaN       NaN       NaN        NaN   
2022-08-29       NaN       NaN       NaN       NaN       NaN        NaN   
2022-08-30       NaN       NaN       NaN       NaN       NaN        NaN   
2022-08-31       NaN       NaN       NaN       NaN       NaN        NaN   

CURTYP                            
bank                              
KIND                              
Bucket     1460-1825D 1825-2190D  
2015-10-31        0.0        0.0  
2015-11-01        0.0        0.0  
2015-11-02        0.0        0.0  
2015-11-03        0.0        0.0  
2015-11-04        0.0        0.0  
...               ...        ...  
2022-08-27        NaN        NaN  
2022-08-28        NaN        NaN  
2022-08-29        NaN        NaN  
2022-08-30        NaN        NaN  
2022-08-31        NaN        NaN  

[2497 rows x 58 columns]

In [317]:
# Biến tỷ lệ rút (biến phụ thuộc)
Withdraw_Ratio=pd.read_excel(r"C:\Users\ngocna3\Documents\My Received Files\Ratio_final.xlsx", header = [0,1,2,3], index_col = [0])
wd_reg=Withdraw_Ratio.loc[(Withdraw_Ratio.index>=start_day_reg) & (Withdraw_Ratio.index<=end_day_reg),
((Withdraw_Ratio.columns.get_level_values("CURTYP")=="VND") & (Withdraw_Ratio.columns.get_level_values("bank").isin(["RC"]))) ]
#wd_reg.columns=["{}_{}_WD_{}".format(i[0],i[1],i[3]) for i in wd_reg.columns]

In [320]:
wd_reg.columns = ["{}_{}_WD_{}".format(i[0],i[1], i[3]) for i in wd_reg.columns]

In [321]:
wd_reg

,VND_RC_WD_0-30,VND_RC_WD_30-90,VND_RC_WD_90-180,VND_RC_WD_180-365,VND_RC_WD_365-730,VND_RC_WD_730-1095,VND_RC_WD_1095-1460,VND_RC_WD_1825-2190,VND_RC_WD_1460-1825
2018-07-02,0.015092,0.006141,0.009348,0.009990,0.003566,0.0,0.0,NaN,NaN
2018-07-03,0.015120,0.006373,0.009227,0.010016,0.003857,0.0,0.0,NaN,NaN
2018-07-04,0.016331,0.006419,0.008904,0.009834,0.003385,0.0,0.0,NaN,NaN
2018-07-05,0.015359,0.006518,0.008632,0.009862,0.003706,0.0,0.0,NaN,NaN
2018-07-06,0.016748,0.006723,0.008750,0.011761,0.002607,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2022-08-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
wd_reg_use.VND_RC_WD_1095.unique()

#### Biến lãi suất

In [323]:
def clean_num (df, col_list):
    for i in col_list:
        if df[i].dtypes == np.dtype("O"):
            df[i] = pd.to_numeric(df[i])
    return df

In [324]:
#```python
# Xử lý dữ liệu thô lãi suất : file IR Survey.xlsx
st_date="2018-07-02"
end_date="2022-08-31"
# XỬ LÝ IR
IR=pd.read_excel(path.join(data_folder,"IR Survey_upto_310822.xlsx"),sheet_name="RAW")
# drop duplicated
IR_hanlded=pd.DataFrame()
for i in IR.BANK.unique():
    IR_bank=IR[IR.BANK==i].copy().ffill().drop_duplicates(IR.columns[:-1])
    IR_bank.loc[IR_bank.Date.duplicated(),"Date"]=IR_bank.loc[IR_bank.Date.duplicated(),"Report_Date"]
    IR_hanlded=IR_hanlded.append(IR_bank)
IR_hanlded['Date'] = pd.to_datetime(IR_hanlded['Date'], errors = 'coerce')
# pivot_table to fill NaN
IR=IR_hanlded.pivot_table(values=IR_hanlded.columns[5:-1],columns="BANK",index="Date").reorder_levels([1,0],axis=1).sort_index(axis=1)
IR=IR.reindex(pd.date_range(IR.index.min(),IR.index.max())).ffill().reindex(pd.date_range(st_date,end_date)).dropna(how="all").reset_index()
# Melt table
IR=IR.melt("index",var_name=["Bank","Term"],value_name="rate").pivot_table("rate",["index","Bank"],"Term")
# Interpolate missing value (tpbank 15M)
IR.columns=IR.columns.map(lambda x:int(x[:-1]))
IR=IR.sort_index(axis=1).reset_index().rename(columns={"index":"day"})
# set group bank 
G1=["ACB","EXIMBANK","MB","SACOMBANK","TECHCOMBANK","VIB"]
G2=["LIENVIET BANK","DONG A BANK","PVCOMBANK","SCB","SHB","SEABANK","TP BANK","VP BANK"]
IR["group"]=IR.Bank.map(lambda x: "G1" if x.upper() in G1 else ("G2" if x.upper() in G2 else "MSB"))
# groupby by Date and Group
IR_groupby_term=IR.groupby(["day","group"],as_index=False).agg(np.nanmean)
IR_groupby_term.columns=IR_groupby_term.columns.map(lambda x: x if type(x) == str else str(x)+"M")
IR_groupby_term.to_pickle(path.join(data_folder,"IR_groupby_term.pickle"))
#```

In [325]:
# Bien lai suat
IR_groupby_term=pd.read_pickle(data_folder+r"\IR_groupby_term.pickle")

# Group Term LS
IR_groupby_term["Short_Term"]=(IR_groupby_term["1M"]+IR_groupby_term["3M"])/2
IR_groupby_term["Medium_Term"]=(IR_groupby_term["6M"]+IR_groupby_term["9M"])/2
IR_groupby_term["Long_Term"]=(IR_groupby_term["12M"]+IR_groupby_term["13M"]+IR_groupby_term["15M"]+
                              IR_groupby_term["18M"]+IR_groupby_term["24M"]+IR_groupby_term["36M"])/6

G1=IR_groupby_term[IR_groupby_term.group=="G1"].set_index("day").drop("group",axis=1)
G2=IR_groupby_term[IR_groupby_term.group=="G2"].set_index("day").drop("group",axis=1)
MSB=IR_groupby_term[IR_groupby_term.group=="MSB"].set_index("day").drop("group",axis=1)

MSB_G1=MSB-G1
MSB_G2=MSB-G2
MSB.columns="MSB_"+MSB.columns
MSB_G1.columns="MSB_G1_"+MSB_G1.columns
MSB_G2.columns="MSB_G2_"+MSB_G2.columns

IR_variable=pd.concat([MSB,MSB_G1,MSB_G2],axis=1)
IR_variable.index.name="Date"
IR_variable=IR_variable.reindex(wd_reg.index).ffill()

#### biến dummy condition period

In [326]:
# biến dummy condition period
dummy_period_variables=pd.DataFrame(columns=["ss_dummy","covid_dummy"],index=wd_reg.index)
dummy_period_variables["ss_dummy"]=dummy_period_variables.index.isin(ss_day_t0).astype(int)
dummy_period_variables["covid_dummy"]=dummy_period_variables.index.isin(covid).astype(int)

In [327]:
dummy_period_variables.reset_index(inplace = True)
dummy_period_variables = dummy_period_variables.rename(columns = {'index':'Date'})

In [328]:
dummy_period_variables

,Date,ss_dummy,covid_dummy
0,2018-07-02,0,0
1,2018-07-03,0,0
2,2018-07-04,0,0
3,2018-07-05,0,0
4,2018-07-06,0,0
...,...,...,...
1517,2022-08-27,0,0
1518,2022-08-28,0,0
1519,2022-08-29,0,0
1520,2022-08-30,0,0


MEF

In [329]:
mef = pd.read_excel(path.join(data_folder,'mef_irrbb.xlsx'))
mef.Date = pd.to_datetime(mef.Date, errors = 'coerce')
df_add = pd.DataFrame(columns = mef.columns[1:], data = np.nan, index = pd.date_range(st_date,end_date)).reset_index().rename(columns = {'index':'Date'})
mef_new = df_add.append(mef)
mef_new.sort_values('Date', ascending = True, inplace = True)
mef_final = mef_new.fillna(method = 'ffill')
mef_final = mef_final[(mef_final.Date >= st_date) & (mef_final.Date <= end_date)]

In [330]:
IR_variable

Term,MSB_1M,MSB_3M,MSB_6M,MSB_9M,MSB_12M,MSB_13M,MSB_15M,MSB_18M,MSB_24M,MSB_36M,...,MSB_G2_9M,MSB_G2_12M,MSB_G2_13M,MSB_G2_15M,MSB_G2_18M,MSB_G2_24M,MSB_G2_36M,MSB_G2_Short_Term,MSB_G2_Medium_Term,MSB_G2_Long_Term
2018-07-02,5.158167,5.375,6.90,6.9,6.90,6.95,7.15,6.95,6.85,6.85,...,0.131250,-0.406250,-0.456250,-0.287500,-0.50625,-0.825000,-0.900000,0.085333,0.221875,-0.563542
2018-07-03,5.158167,5.375,6.90,6.9,6.90,6.95,7.15,6.95,6.85,6.85,...,0.131250,-0.406250,-0.456250,-0.287500,-0.50625,-0.825000,-0.900000,0.085333,0.221875,-0.563542
2018-07-04,5.158167,5.375,6.90,6.9,6.90,6.95,7.15,6.95,6.85,6.85,...,0.131250,-0.406250,-0.456250,-0.287500,-0.50625,-0.825000,-0.900000,0.085333,0.221875,-0.563542
2018-07-05,5.158167,5.375,6.90,6.9,6.90,6.95,7.15,6.95,6.85,6.85,...,0.131250,-0.406250,-0.456250,-0.287500,-0.50625,-0.825000,-0.900000,0.085333,0.221875,-0.563542
2018-07-06,5.158167,5.375,6.90,6.9,6.90,6.95,7.15,6.95,6.85,6.85,...,0.131250,-0.406250,-0.456250,-0.287500,-0.50625,-0.825000,-0.900000,0.085333,0.221875,-0.563542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-27,3.700000,3.900,5.25,5.2,5.75,5.75,5.75,5.75,5.75,5.75,...,-0.675417,-0.649375,0.166458,-0.892222,-0.83125,-0.857708,-1.065208,0.070000,-0.580104,-0.688218
2022-08-28,3.700000,3.900,5.25,5.2,5.75,5.75,5.75,5.75,5.75,5.75,...,-0.675417,-0.649375,0.166458,-0.892222,-0.83125,-0.857708,-1.065208,0.070000,-0.580104,-0.688218
2022-08-29,3.700000,3.900,5.25,5.2,5.75,5.75,5.75,5.75,5.75,5.75,...,-0.675417,-0.649375,0.166458,-0.892222,-0.83125,-0.857708,-1.065208,0.070000,-0.580104,-0.688218
2022-08-30,3.700000,3.900,5.25,5.2,5.75,5.75,5.75,5.75,5.75,5.75,...,-0.675417,-0.649375,0.166458,-0.892222,-0.83125,-0.857708,-1.065208,0.070000,-0.580104,-0.688218


In [331]:
IR_variable.reset_index(inplace = True)

In [332]:
IR_variable

Term,index,MSB_1M,MSB_3M,MSB_6M,MSB_9M,MSB_12M,MSB_13M,MSB_15M,MSB_18M,MSB_24M,...,MSB_G2_9M,MSB_G2_12M,MSB_G2_13M,MSB_G2_15M,MSB_G2_18M,MSB_G2_24M,MSB_G2_36M,MSB_G2_Short_Term,MSB_G2_Medium_Term,MSB_G2_Long_Term
0,2018-07-02,5.158167,5.375,6.90,6.9,6.90,6.95,7.15,6.95,6.85,...,0.131250,-0.406250,-0.456250,-0.287500,-0.50625,-0.825000,-0.900000,0.085333,0.221875,-0.563542
1,2018-07-03,5.158167,5.375,6.90,6.9,6.90,6.95,7.15,6.95,6.85,...,0.131250,-0.406250,-0.456250,-0.287500,-0.50625,-0.825000,-0.900000,0.085333,0.221875,-0.563542
2,2018-07-04,5.158167,5.375,6.90,6.9,6.90,6.95,7.15,6.95,6.85,...,0.131250,-0.406250,-0.456250,-0.287500,-0.50625,-0.825000,-0.900000,0.085333,0.221875,-0.563542
3,2018-07-05,5.158167,5.375,6.90,6.9,6.90,6.95,7.15,6.95,6.85,...,0.131250,-0.406250,-0.456250,-0.287500,-0.50625,-0.825000,-0.900000,0.085333,0.221875,-0.563542
4,2018-07-06,5.158167,5.375,6.90,6.9,6.90,6.95,7.15,6.95,6.85,...,0.131250,-0.406250,-0.456250,-0.287500,-0.50625,-0.825000,-0.900000,0.085333,0.221875,-0.563542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1517,2022-08-27,3.700000,3.900,5.25,5.2,5.75,5.75,5.75,5.75,5.75,...,-0.675417,-0.649375,0.166458,-0.892222,-0.83125,-0.857708,-1.065208,0.070000,-0.580104,-0.688218
1518,2022-08-28,3.700000,3.900,5.25,5.2,5.75,5.75,5.75,5.75,5.75,...,-0.675417,-0.649375,0.166458,-0.892222,-0.83125,-0.857708,-1.065208,0.070000,-0.580104,-0.688218
1519,2022-08-29,3.700000,3.900,5.25,5.2,5.75,5.75,5.75,5.75,5.75,...,-0.675417,-0.649375,0.166458,-0.892222,-0.83125,-0.857708,-1.065208,0.070000,-0.580104,-0.688218
1520,2022-08-30,3.700000,3.900,5.25,5.2,5.75,5.75,5.75,5.75,5.75,...,-0.675417,-0.649375,0.166458,-0.892222,-0.83125,-0.857708,-1.065208,0.070000,-0.580104,-0.688218


In [333]:
IR_variable.rename(columns = {'index':'Date'}, inplace = True)

In [334]:
wd_reg.reset_index(inplace = True)

In [335]:
wd_reg = wd_reg.rename(columns = {'index':'Date'})

In [336]:
data_reg = wd_reg.merge((IR_variable.merge(mef_final, how = 'left', on = ['Date'])).
                merge(dummy_period_variables, how = 'left', on = ['Date']), how = 'left', on = ['Date'])

In [337]:
data_reg

,Date,VND_RC_WD_0-30,VND_RC_WD_30-90,VND_RC_WD_90-180,VND_RC_WD_180-365,VND_RC_WD_365-730,VND_RC_WD_730-1095,VND_RC_WD_1095-1460,VND_RC_WD_1825-2190,VND_RC_WD_1460-1825,...,GDP_lead_6m,GDP_lead_9m,GDP_lead_12m,GDP_per_change,GDP_ma_3m,GDP_ma_6m,GDP_ma_9m,GDP_ma_12m,ss_dummy,covid_dummy
0,2018-07-02,0.015092,0.006141,0.009348,0.009990,0.003566,0.0,0.0,NaN,NaN,...,7.31,6.82,6.73,-0.040650,7.08,7.230,7.37,7.3925,0,0
1,2018-07-03,0.015120,0.006373,0.009227,0.010016,0.003857,0.0,0.0,NaN,NaN,...,7.31,6.82,6.73,-0.040650,7.08,7.230,7.37,7.3925,0,0
2,2018-07-04,0.016331,0.006419,0.008904,0.009834,0.003385,0.0,0.0,NaN,NaN,...,7.31,6.82,6.73,-0.040650,7.08,7.230,7.37,7.3925,0,0
3,2018-07-05,0.015359,0.006518,0.008632,0.009862,0.003706,0.0,0.0,NaN,NaN,...,7.31,6.82,6.73,-0.040650,7.08,7.230,7.37,7.3925,0,0
4,2018-07-06,0.016748,0.006723,0.008750,0.011761,0.002607,0.0,0.0,NaN,NaN,...,7.31,6.82,6.73,-0.040650,7.08,7.230,7.37,7.3925,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1533,2022-08-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.65,6.15,6.40,0.534791,7.72,6.375,5.99,2.9500,0,0
1534,2022-08-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.65,6.15,6.40,0.534791,7.72,6.375,5.99,2.9500,0,0
1535,2022-08-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.65,6.15,6.40,0.534791,7.72,6.375,5.99,2.9500,0,0
1536,2022-08-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.65,6.15,6.40,0.534791,7.72,6.375,5.99,2.9500,0,0


In [ ]:
IR_variable = IR_variable[IR_variable.columns[2:].tolist()]

### Phân tích hồi quy

#### Function

In [24]:
def filter_IR_with_correlation(IR_variable,test_size=0,max_corr=0.65):
    IR_corr_matrix=IR_variable.iloc[:(None if test_size==0 else -test_size)].corr()
    # lựa chọn tập X trên threshold correlation
    sign=(IR_corr_matrix.abs()<max_corr).astype(int)
    list_variables=IR_variable.columns.tolist()
    list_X_feature=[[i] for i in list_variables]
    for n in range(2, len(list_variables)+1):
        check_n_1=[i for i in list_X_feature if len(i)==(n-1)]
        if len(check_n_1)==0:
            break
        list_X_feature_n=[]
        for a in check_n_1:
            list_X_feature_n+=[np.sort(a+[x]).tolist() for x in sign[sign[a].prod(1)==1].index]
            # maximum 4 variables (include intercept)
            
            list_X_feature_n= [i for i in list_X_feature_n if len(i)<5] 
        list_X_feature+=pd.DataFrame(list_X_feature_n).drop_duplicates().values.tolist()
    return list_X_feature
#IR_list=[None]+filter_IR_with_correlation(IR_variable,test_size=0,max_corr=0.5)

In [ ]:
all_var = data_reg[data_reg.columns[7:]]

In [ ]:
list_var = [None]+filter_IR_with_correlation(all_var,test_size=0,max_corr=0.5)

In [25]:
def handling_data_plot(df,tw,c="VND",b="RB",per=None,test_size=test_size,type_adjust=["max"]):
    df=df.copy()
    x=df.index
    x_end_train=x[-test_size]
    df.columns=["Real","Predict"]
    
    # Cách hiệu chỉnh
    for type_adjust in ["max","normal","seasonal"]:
        if type_adjust=="max":
            adjust_reg=(df["Real"]-df["Predict"]).loc[:x_end_train-pd.Timedelta("1D")].max()
            df["Predict_adjust_{}".format(type_adjust)]=df["Predict"]+adjust_reg
            print("Giá trị hiệu chỉnh {} (%) = ".format(type_adjust),round(adjust_reg,2))
        if type_adjust=="normal":
            percentile_adjust=90
            adjust_reg=np.nanpercentile((df["Real"]-df["Predict"]).loc[df.index.isin(ss_day_t0)==False].dropna().loc[:x_end_train-pd.Timedelta("1D")].iloc[-365:],percentile_adjust)
            df["Predict_adjust_{}".format(type_adjust)]=df["Predict"]+adjust_reg
            print("Giá trị hiệu chỉnh {} (%) = ".format(type_adjust),round(adjust_reg,2))
        if type_adjust=="seasonal":
            percentile_adjust=90
            adjust_reg=np.nanpercentile((df["Real"]-df["Predict"]).loc[df.index.isin(ss_day_t0)].dropna().loc[:x_end_train-pd.Timedelta("1D")],percentile_adjust)
            df["Predict_adjust_{}".format(type_adjust)]=df["Predict"]+adjust_reg
            print("Giá trị hiệu chỉnh {} (%) = ".format(type_adjust),round(adjust_reg,2))
    #     if type_adjust=="stress":
    #         percentile_adjust=-10
    #         adjust_reg=np.sort((df["Real"]-df["Predict"]).dropna().loc[:x_end_train-pd.Timedelta("1D")])[percentile_adjust]
    #         df["Predict_adjust_{}".format(type_adjust)]=df["Predict"]+adjust_reg
    df["mix_regression"]=(df.index.isin(st_msb)*df["Predict_adjust_max"])+\
        ((df.index.isin(ss_day_t0) & (df.index.isin(st_msb)==False))*df["Predict_adjust_seasonal"])+\
        (((df.index.isin(ss_day_t0)==False) & (df.index.isin(st_msb)==False))*df["Predict_adjust_normal"])
    
    
    if per is not None:
        per_filter=per.loc[(per.CURTYP==c) & (per.bank==b)].set_index("Time").loc[:,("WD_PRE_RATIO",tw)]
        per_filter.values
        df[per_filter.index]=per_filter.values*100
        df["mix_percentile"]=(df.index.isin(st_msb)*df.Stress)+\
        ((df.index.isin(ss_day_t0) & (df.index.isin(st_msb)==False))*df.Seasonal)+\
        (((df.index.isin(ss_day_t0)==False) & (df.index.isin(st_msb)==False))*df.Normal)

    return df

In [26]:
def plot_reg(df,tw,test_size,b,c,figsize=(10, 4),dpi=100):
    x=df.index
    x_end_train=x[-test_size]
    fig, ax = plt.subplots(1, figsize=figsize,dpi=dpi)
    ax.set_title("Mô hình hồi quy danh mục {} {} Time-window {} ngày".format(b,c,tw).upper())
    ax.grid(alpha=0.3)

    ax.fill_between(x, 0, df["Real"], alpha=0.1,color="gray")
    ax.fill_between(x, 0,df.loc[df.index.isin(covid),"Real"].reindex(x),color="orange", alpha=0.2,label="Covid Period",hatch='.')
    ax.fill_between(x, 0,df.loc[df.index.isin(ss_day_t0),"Real"].reindex(x),color="violet", alpha=0.3,label="Seasonal Period")
    
    ax.plot(x,df["Predict"],color="goldenrod",label="Tỷ lệ Rút hồi quy",alpha=0.7,linewidth=0.7)
    
    ax.plot(x,(df["Predict_adjust_normal"]),"g-",label="Tỷ lệ rút normal regression",alpha=0.7,linewidth=0.7)
    ax.plot(x,(df["Predict_adjust_seasonal"]),"b-",label="Tỷ lệ rút seasonal regression",alpha=0.7,linewidth=0.7)
#     ax.plot(x,(df["Predict_adjust_max"]),color="violet",ls="-",label="Tỷ lệ rút stress regression",alpha=0.7,linewidth=0.7)
    
    ax.plot(x,df["Real"],color="gray",label="Tỷ lệ Rút thực tế")
    ax.plot(x,(df["mix_regression"]),ls="-",color="orangered",label="Tỷ lệ rút hồi quy hiệu chỉnh",linewidth=1.5) 

    ax.legend(ncol=3,loc="upper left")
    ax.vlines(x=x_end_train, ymin=0, ymax=df.max().max()*1.2,ls="--",color="green",linewidth=2)
    ax.annotate(text="Test Period",xy=(x_end_train+pd.Timedelta("3D"),df.max().max()))
#     ax.set_ylim( 0, df.max().max()*1.2)
    ax.set_ylabel("%")
    ax.set_xlabel('Begin date')
    return fig

In [27]:
def plot_mix(df,tw,test_size,b,c,figsize=(10, 4),dpi=100):
    x=df.index
    x_end_train=x[-test_size]
    fig, ax = plt.subplots(1, figsize=figsize,dpi=dpi)
    ax.set_title("So sánh mô hình hồi quy và percentile danh mục {} {} Time-window {} ngày".format(b,c,tw).upper())
    ax.grid(alpha=0.3)
        
    ax.plot(x,(df["Predict"]),ls="-",color="deepskyblue",label="Tỷ lệ rút hồi quy",alpha=0.7,linewidth=0.7) # olive
    
    ax.plot(x,df["Real"],color="gray",label="Tỷ lệ Rút thực tế")
    ax.fill_between(x, 0, df["Real"], alpha=0.1,color="gray")
    ax.fill_between(x, 0,df.loc[df.index.isin(covid),"Real"].reindex(x),color="orange", alpha=0.2,label="Covid Period",hatch='.')
    ax.fill_between(x, 0,df.loc[df.index.isin(ss_day_t0),"Real"].reindex(x),color="violet", alpha=0.3,label="Seasonal Period")
    
    ax.plot(x,(df["mix_regression"]),ls="-",color="orangered",label="Tỷ lệ rút hồi quy sau hiệu chỉnh",alpha=0.7) # olive
    ax.plot(x,(df["Normal"]),"y--",label="Tỷ lệ rút percentile normal",alpha=0.7)
    ax.plot(x,(df["Seasonal"]),"g--",label="Tỷ lệ rút percentile seasonal",alpha=0.7)
    ax.plot(x,(df["Stress"]),"b--",label="Tỷ lệ rút percentile stress",alpha=0.7)
    
    
    
    ax.legend(ncol=3,loc="upper left")
    ax.vlines(x=x_end_train, ymin=0, ymax=df.max().max()*1.4,ls="--",color="green",linewidth=2)
    ax.annotate(text="Test Period",xy=(x_end_train+pd.Timedelta("3D"),df.max().max()))
    ax.set_ylim( 0, df.max().max()*1.4)
    ax.set_ylabel("%")
    ax.set_xlabel('Begin date')
    return fig


In [28]:
def performance_cal(df):
    # RMSE chỉ lấy abs, ko có relative
    
    x=[i for i in df.columns if "mix" in i]
    df[[i+"_diff_abs" for i in x]]=df[x].values-df[["Real"]].values
    df[[i+"_diff_rel" for i in x]]=(df[x].values-df[["Real"]].values)/df[["Real"]].values
    df_diff=df[[i for i in df.columns if "diff" in i]]
    performance=pd.concat([(df_diff.pow(2).sum()/len(df_diff)).pow(0.5),
    df_diff.abs().max(),
    df_diff.mean(),
    df_diff.abs().min()],axis=1)
    performance.columns=["RMSE","max_diff","mean_diff","min_diff"]
    performance=performance.T
    for i in performance.columns:
        performance.loc["RMSE",i]=performance.loc["RMSE",i.replace("rel","abs")]
    return performance

In [ ]:
wd_reg.reset_index(inplace = True)

In [ ]:
wd_reg.rename(columns = {'index':'Date'}, inplace = True)

In [ ]:
data_reg = wd_reg.merge(all_variables, how = 'left', on = ['Date'])

In [ ]:
data_reg.columns[1:6].tolist()

In [29]:
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

In [30]:
def get_dataframe_column_values(df, col_idx):
    col_data = df.iloc[:,col_idx:].values
    res1 = []
    res2 = []
    for a in col_data:
        res1.append(a[0])
        if len(a)>1:
            res2.append(a[1])
    return np.array(res1), np.array(res2)

In [31]:
all_var  = IR_variable.merge(mef_final, how = 'left', on = ['Date'])

In [32]:
all_var.head()

,Date,MSB_1M,MSB_3M,MSB_6M,MSB_9M,MSB_12M,MSB_13M,MSB_15M,MSB_18M,MSB_24M,...,GDP_lag_12m,GDP_lead_3m,GDP_lead_6m,GDP_lead_9m,GDP_lead_12m,GDP_per_change,GDP_ma_3m,GDP_ma_6m,GDP_ma_9m,GDP_ma_12m
0,2018-07-02,5.158167,5.375,6.9,6.9,6.9,6.95,7.15,6.95,6.85,...,6.28,6.88,7.31,6.82,6.73,-0.04065,7.08,7.23,7.37,7.3925
1,2018-07-03,5.158167,5.375,6.9,6.9,6.9,6.95,7.15,6.95,6.85,...,6.28,6.88,7.31,6.82,6.73,-0.04065,7.08,7.23,7.37,7.3925
2,2018-07-04,5.158167,5.375,6.9,6.9,6.9,6.95,7.15,6.95,6.85,...,6.28,6.88,7.31,6.82,6.73,-0.04065,7.08,7.23,7.37,7.3925
3,2018-07-05,5.158167,5.375,6.9,6.9,6.9,6.95,7.15,6.95,6.85,...,6.28,6.88,7.31,6.82,6.73,-0.04065,7.08,7.23,7.37,7.3925
4,2018-07-06,5.158167,5.375,6.9,6.9,6.9,6.95,7.15,6.95,6.85,...,6.28,6.88,7.31,6.82,6.73,-0.04065,7.08,7.23,7.37,7.3925


In [33]:
def loc_bien(b,c,bk, wd_reg, all_var):
    Y_var = wd_reg[["{}_{}_WD_{}".format(c,b,bk)]]
    df = pd.DataFrame()
    for col in all_var.columns.tolist():
        corre = all_var[col].corr(Y_var[Y_var.columns[0]])
        corr_df = pd.DataFrame(data = {Y_var.columns[0] :[corre]}, index = [col])
        df = df.append(corr_df)
    df = df[(df[Y_var.columns[0]].abs() > 0.4)]
    df.reset_index(inplace = True)
    all_x_var_list = df['index'].unique().tolist()
    df_reg = pd.concat([Y_var,all_var[[i for i in all_var.columns.tolist() if i in all_x_var_list]]], axis = 1)
    return df_reg

In [338]:
data_reg.head()

,Date,VND_RC_WD_0-30,VND_RC_WD_30-90,VND_RC_WD_90-180,VND_RC_WD_180-365,VND_RC_WD_365-730,VND_RC_WD_730-1095,VND_RC_WD_1095-1460,VND_RC_WD_1825-2190,VND_RC_WD_1460-1825,...,GDP_lead_6m,GDP_lead_9m,GDP_lead_12m,GDP_per_change,GDP_ma_3m,GDP_ma_6m,GDP_ma_9m,GDP_ma_12m,ss_dummy,covid_dummy
0,2018-07-02,0.015092,0.006141,0.009348,0.009990,0.003566,0.0,0.0,NaN,NaN,...,7.31,6.82,6.73,-0.04065,7.08,7.23,7.37,7.3925,0,0
1,2018-07-03,0.015120,0.006373,0.009227,0.010016,0.003857,0.0,0.0,NaN,NaN,...,7.31,6.82,6.73,-0.04065,7.08,7.23,7.37,7.3925,0,0
2,2018-07-04,0.016331,0.006419,0.008904,0.009834,0.003385,0.0,0.0,NaN,NaN,...,7.31,6.82,6.73,-0.04065,7.08,7.23,7.37,7.3925,0,0
3,2018-07-05,0.015359,0.006518,0.008632,0.009862,0.003706,0.0,0.0,NaN,NaN,...,7.31,6.82,6.73,-0.04065,7.08,7.23,7.37,7.3925,0,0
4,2018-07-06,0.016748,0.006723,0.008750,0.011761,0.002607,0.0,0.0,NaN,NaN,...,7.31,6.82,6.73,-0.04065,7.08,7.23,7.37,7.3925,0,0


In [ ]:
["{}_{}_WD_{}".format(i[0],i[1], i[3]) for i in wd_reg.columns]

In [42]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import statsmodels.stats.api as sms
from statsmodels.formula.api import ols
from scipy.stats import kstest
from scipy.stats import ks_2samp
from statsmodels.stats.stattools import durbin_watson
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import het_white
from sklearn.metrics import mean_absolute_percentage_error
import scipy.stats as stats
from IPython.display import clear_output




def data_split(data,y,test_size=4):
    if test_size<=1:
        point_split=round((1-test_size)*len(data))
    else:
        point_split=int(len(data)-test_size)
    x_train=data[[i for i in data.columns if i != y]].iloc[:point_split].copy()
    y_train=data[[y]].iloc[:point_split].copy()
    x_test=data[[i for i in data.columns if i != y]].iloc[point_split:].copy()
    y_test=data[[y]].iloc[point_split:].copy()
    return x_train,y_train,x_test,y_test



def run_OLS(x_name, y_name,data,poly_degree=1,return_plot=False,test_size=30,return_summary=True):
    x_name=x_name.copy()
    if return_summary:
        global real_y,predict_y,predict_y_test
        
    x_name=[x_name] if type(x_name)==str else x_name
    number_var=len(x_name)
          
    data=data[[y_name]+x_name].copy().dropna()

    poly=PolynomialFeatures(degree=poly_degree)
    data_x=pd.DataFrame(poly.fit_transform(data[x_name]),columns=poly.get_feature_names(input_features=x_name),index=data.index)
    data=pd.concat([data[y_name],data_x],axis=1).dropna()
    data1=pd.DataFrame(data)
    
    x_train,y_train,x_test,y_test=data_split(data,y_name,test_size=test_size)
#     print(len(y_train.values), len(x_train))
    res = sm.OLS(y_train.values,x_train ).fit()
    predict_y_test = res.predict(x_test)
    all_y = y_train.append(y_test, ignore_index = True)
    all_x = x_train.append(x_test, ignore_index = True)
#     all_x = pd.concat([x_train, x_test],axis = 0)
    predict_all_y = res.predict(all_x)
    RMSE= np.nan if test_size==1 else np.sqrt(mean_squared_error(all_y,predict_all_y)) # Root Mean Square Error(RMSE)
    all_res = sm.OLS(all_y.values, all_x).fit()
#     MAPE_all = mape(all_y, predict_all_y)
    MAPE_all_new = mean_absolute_percentage_error(all_y, predict_all_y)

    
    #Tinh cook distance
    influence = res.get_influence()
    cooks = influence.cooks_distance

    vif = [variance_inflation_factor(data_x.values, i) 
               for i in range(data_x.shape[1])]
    
    #Tinh MAPE
    MAPE_result = mape(y_test,predict_y_test)

    if return_plot==False: 
        
        num_inappropriate_pvalue=int(sum(res.pvalues>0.05))
        f_pvalue=res.f_pvalue
        cooks = res.get_influence().cooks_distance
        D_W = durbin_watson(res.resid)
        
        #Tinh residuals
        residuals = np.array(y_test) - np.array(predict_y_test)
         # Calculate ks
        ks_data_x1, ks_data_x2 = get_dataframe_column_values(data1, 2)
        
        ks_data_y, _ = get_dataframe_column_values(data1, 0)
        
        ks1 = ks_2samp(ks_data_x1, ks_data_y)
        ks2 = ks1
        
        ks3 = kstest(res.resid,'norm', alternative = 'greater')
        #Tinh Dickey_fuller
        ADF_y = adfuller(ks_data_y)
        ADF_x1 = adfuller(ks_data_x1)
        ADF_x2 = ADF_x1
        ADF_resid_AIC = adfuller(res.resid, autolag='AIC')
        ADF_resid_BIC = adfuller(res.resid, autolag='BIC')

        if len(ks_data_x2) > 0:
            ks2 = ks_2samp(ks_data_x2, ks_data_y)
            ADF_x2 = adfuller(ks_data_x2)
            
        ##Count %Cooks Value Pass
        result = []
        for i in range(len(cooks[0].tolist())):
            if cooks[0].tolist()[i] <= 4/len(data1):
                result.append(cooks[0].tolist()[i])
            i = i + 1
        result = len(result)/len(cooks[0].tolist()) 
        
#         Tinh White_het
        White=0
        try:
            white_test = het_white(res.resid,  res.model.exog)
            labels = ['Test Statistic', 'Test Statistic p-value', 'F-Statistic', 'F-Test p-value']
            WhiteObj = dict(zip(labels, white_test))
            White = WhiteObj['Test Statistic p-value']
        except:
            White = {"Error":"AssertionErr"}
        
        df_result=pd.DataFrame().reindex(x_train.columns)
        df_result=pd.DataFrame(
            [y_name,res.params.index.tolist(),number_var+1,poly_degree,num_inappropriate_pvalue,
             res.rsquared,res.rsquared_adj, res.pvalues.tolist(), f_pvalue,RMSE,res.params.tolist(),
             cooks[0].max(), result,D_W,vif, ks3.pvalue, ADF_resid_AIC[1], ADF_resid_BIC[1],White,
            MAPE_all_new],
            index=["Y_feature","X_feature","number_var","degree_poly","num_inappropriate_pvalue","R_2","Adj_R_2",
                   "P_value", "f_pvalue","RMSE","Coef","Cooks's Distance Max","%Cooks Value Pass","Durbin Watson","VIF", 
                   "KS test", "ADF test AIC", "ADF test BIC","White_test",  "MAPE of all"]).T
        return df_result
    else:
        x = data.index
        real_y = data[y_name]
        predict_y = res.get_prediction(data_x).summary_frame(alpha=0).iloc[:,0]
        fig,ax  = plt.subplots(1,figsize=(10,4))
        ax.plot(x,real_y)
        ax.plot(x,predict_y,lw=1.5)
        ax.plot(y_test.index,predict_y_test, 'g',lw=1.5)
        ax.legend(["Real","Estimate","Predict"])
        plt.show()
        
        if return_summary:
            print(RMSE)
#             print(f'MAPE logit new {MAPE_all_logit_new}')
#             print(f'MAPE ODR new {MAPE_all_ODR_new}')
            return res.summary()

In [339]:
data_reg.columns

Index(['Date', 'VND_RC_WD_0-30', 'VND_RC_WD_30-90', 'VND_RC_WD_90-180',
       'VND_RC_WD_180-365', 'VND_RC_WD_365-730', 'VND_RC_WD_730-1095',
       'VND_RC_WD_1095-1460', 'VND_RC_WD_1825-2190', 'VND_RC_WD_1460-1825',
       'MSB_1M', 'MSB_3M', 'MSB_6M', 'MSB_9M', 'MSB_12M', 'MSB_13M', 'MSB_15M',
       'MSB_18M', 'MSB_24M', 'MSB_36M', 'MSB_Short_Term', 'MSB_Medium_Term',
       'MSB_Long_Term', 'MSB_G1_1M', 'MSB_G1_3M', 'MSB_G1_6M', 'MSB_G1_9M',
       'MSB_G1_12M', 'MSB_G1_13M', 'MSB_G1_15M', 'MSB_G1_18M', 'MSB_G1_24M',
       'MSB_G1_36M', 'MSB_G1_Short_Term', 'MSB_G1_Medium_Term',
       'MSB_G1_Long_Term', 'MSB_G2_1M', 'MSB_G2_3M', 'MSB_G2_6M', 'MSB_G2_9M',
       'MSB_G2_12M', 'MSB_G2_13M', 'MSB_G2_15M', 'MSB_G2_18M', 'MSB_G2_24M',
       'MSB_G2_36M', 'MSB_G2_Short_Term', 'MSB_G2_Medium_Term',
       'MSB_G2_Long_Term', 'CPI_YoY_Index', 'CPI_YoY_Index_lag_3m',
       'CPI_YoY_Index_lag_6m', 'CPI_YoY_Index_lag_9m', 'CPI_YoY_Index_lag_12m',
       'CPI_YoY_Index_lead_3m', 'CPI_Yo

In [175]:
# return_to = None                 # detail plot_reg plot_mix backtest_mix
(run_OLS(['GDP','MSB_24M','covid_dummy'], 'VND_RC_WD_180',data=data_reg.copy(),poly_degree=1,return_plot=False,test_size=30,return_summary=False))#.to_excel("vnd_090_add.xlsx")

,Y_feature,X_feature,number_var,degree_poly,num_inappropriate_pvalue,R_2,Adj_R_2,P_value,f_pvalue,RMSE,Coef,Cooks's Distance Max,%Cooks Value Pass,Durbin Watson,VIF,KS test,ADF test AIC,ADF test BIC,White_test,MAPE of all
0,VND_RC_WD_180,"[1, GDP, MSB_24M, covid_dummy]",4,1,1,0.496894,0.495752,"[1.499619318028659e-276, 0.15108184566018878, ...",0.0,0.010047,"[0.1317097216805077, 0.00013015728240558768, -...",0.01352,0.96003,0.118202,"[109.37352542291504, 1.4582002106674201, 1.463...",0.0,0.026954,0.011267,0.0,0.19799


In [171]:
data_reg['GDP'].corr(data_reg['VND_RC_WD_365'])

0.2173934651740688

In [1]:
''' hàm OLS để ra y predict và vẽ hình '''
''' new '''
def run_OLS(b,c,tw,ir_names=IR_list[100],dummy_names=["ss_dummy","covid_dummy"],wd_reg=wd_reg,return_to="overview",test_size=test_size,save_fig=False,per=per,figsize=(10, 4),dpi=100):
    y=wd_reg["{}_{}_WD_{}".format(c,b,tw)].iloc[:-(int(tw.split('-')[1].split('D')[0]))].copy()*100
    const_x=pd.DataFrame([[1]]*len(wd_reg.index),index=wd_reg.index,columns=["const"])
    ir_x=None if ((ir_names is None) | (c=="USD")) else IR_variable[ir_names].copy()
    dummy_x=None if (dummy_names is None) else dummy_period_variables[dummy_names].copy()
    
    x_all=pd.concat([const_x,ir_x,dummy_x],axis=1).iloc[:-(int(tw.split('-')[1].split('D')[0]))].copy()
    y_train,y_test,x_train,x_test=train_test_split(y,x_all,test_size=92,shuffle=False)
    
    res = sm.OLS(y_train,x_train ).fit()
    predict_y_test = res.predict(x_test)
    RMSE= np.nan if test_size==1 else mean_squared_error(y_test,predict_y_test,squared=False)

    df_y=pd.concat([y,res.predict(x_all)],axis=1)
    df_y.columns=["Real","Predict"]
    
    df_y_test=pd.concat([y_test,predict_y_test],axis=1)
    df_y_test.columns=["Real_test","Predict_test"] 


    if return_to=="overview":
        return pd.DataFrame(["{}_{}_WD_{}".format(c,b,tw),
                                res.params.index.tolist(),
                                res.params.tolist(),
                                int(sum(res.pvalues>0.05)),
                                res.rsquared,
                                res.f_pvalue,
                                RMSE],
                                index=["Y_feature","X_features","Coef",
                                       "num_unappropriated_pvalue","R_2",
                                       "f_pvalue","RMSE"]).T
    if return_to=="detail":
        print("RMSE trước hiệu chỉnh (%) ",round(RMSE,2))
        print("R2(%) ",round(res.rsquared*100,2))
        print(pd.concat([y,x_all],axis=1).corr()[[y.name]])
        print(res.summary())

    if return_to=="plot_reg":
        df_handled=handling_data_plot(df_y,tw,c=c,b=b,per=None)
        fig=plot_reg(df_handled,tw,test_size,b=b,c=c,figsize=figsize,dpi=dpi)
        if save_fig:
            return fig
        
    if return_to=="plot_mix":
        df_handled=handling_data_plot(df_y,tw,c=c,b=b,per=per)
        fig=plot_mix(df_handled,tw,test_size,b,c,figsize=figsize,dpi=dpi)
        if save_fig:
            return fig
        
    if return_to=="performance":
        df_handled=handling_data_plot(df_y,tw,c=c,b=b,per=per).iloc[-test_size:]/100
        return performance_cal(df_handled).T.round(4)*100
    
    if return_to=="backtest_mix":
        df_handled=handling_data_plot(df_y,tw,c=c,b=b,per=per).rename(columns={"Real":"WD","mix":"Estimate"})/100
        return df_handled
    
    if return_to=="df_y":
        return df_y_test

' hàm OLS để ra y predict và vẽ hình '

In [ ]:
''' hàm define ở trên dùng ở đây '''
b="RC"
c="VND"
bucket = '30'

ir_names= ['MSB_Medium_Term'] #[  'MSB_G2_Medium_Term', 'MSB_Long_Term']
dummy_names=  ['ss_dummy','covid_dummy'] #
# return_to = None                 # detail plot_reg plot_mix backtest_mix
# check_ols_rb = run_OLS(b=b,c=c,tw=bucket,ir_names=ir_names,dummy_names=dummy_names,wd_reg=wd_reg,return_to="df_y",test_size=test_size,save_fig=True,per=per)
run_OLS(b=b,c=c,tw=bucket,ir_names=ir_names,dummy_names=dummy_names,wd_reg=wd_reg,return_to="detail",test_size=test_size,save_fig=True,per=per)
run_OLS(b=b,c=c,tw=bucket,ir_names=ir_names,dummy_names=dummy_names,wd_reg=wd_reg,return_to="plot_mix",test_size=test_size,save_fig=True,per=per)

#### Run loop all model

In [35]:
"""
chạy loop các mô hình theo danh sách list_X_feature
"""

def run_OLS_loop(y_fea,list_X_feature,test_size):
    global res_total
    res_total=pd.DataFrame()
    i=0
    for x_fea in list_X_feature:
        i+=1
        print(i,end="\r")
        res=run_OLS(x_fea.copy(), y_fea,data=df_reg.copy(),poly_degree=1,return_plot=False,test_size=test_size,return_summary=False)
        res_total=res_total.append(res)
    res_total.to_excel('total_models_{}.xlsx'.format(y_fea))
    #res_total.to_excel('./ex_sign_PD_total_models_{}.xlsx'.format(y_fea), sheet_name='Sheet1')

In [366]:
def loc_bien(b,c,bk, wd_reg, all_var):
    Y_var = wd_reg[["{}_{}_WD_{}".format(c,b,bk)]]
    df = pd.DataFrame()
    for col in all_var.columns[1:].tolist():
        corre = all_var[col].corr(Y_var[Y_var.columns[0]])
        corr_df = pd.DataFrame(data = {Y_var.columns[0] :[corre]}, index = [col])
        df = df.append(corr_df)
    df = df[(df[Y_var.columns[0]].abs() > 0.2)]
    df.reset_index(inplace = True)
    all_x_var_list = df['index'].unique().tolist()
    df_reg = pd.concat([Y_var,all_var[[i for i in all_var.columns.tolist() if i in all_x_var_list]]], axis = 1)
    return df_reg

In [360]:
def write_model(df):
    df['num_var'] = df['number_var']-1
    n = df['num_var'].max()
    for i in range(1,n):
        df['X{}'.format(i)] = df['X_feature'].str.split(',').str[i].str.split(']').str[0].str.split("'",n=2, expand = True)[1]
        df['pvalue_X{}'.format(i)] = df['P_value'].str.split(',').str[i].str.split(']').str[0]
        df['VIF_X{}'.format(i)] = df['VIF'].str.split(',').str[i].str.split(']').str[0]
        df['coef_X{}'.format(i)] = df['Coef'].str.split(',').str[i].str.split(']').str[0]
    df['pvalue_intercept'] = df['P_value'].str.split(',').str[0].str.split('[').str[1]
    df['VIF_intercept'] = df['VIF'].str.split(',').str[0].str.split('[').str[1]
    df['coef_intercept'] = df['Coef'].str.split(',').str[0].str.split('[').str[1]
    df_final =df[['Y_feature'] + ['X{}'.format(i) for i in range(1,n)] 
                 + ['coef_intercept'] + ['coef_X{}'.format(i) for i in range(1,n)] +
                 ['pvalue_intercept'] + ['pvalue_X{}'.format(i) for i in range(1,n)] 
                + ['VIF_X{}'.format(i) for i in range(1,n)]
                + ['num_var','R_2','MAPE of all','Adj_R_2', 'VIF_intercept',"Cooks's Distance Max", 
                    '%Cooks Value Pass',
                    'Durbin Watson', 'KS test', 'ADF test AIC', 'ADF test BIC',
                    'White_test', 'MAPE of all']]
    col_list = df_final.columns.tolist()
    for i in col_list[6:]:
        df_final[i] = df_final[i].astype('float64')
    #template = filter_model(df_final)
    df_final.to_excel('./final_template{}_{}_WD_{}.xlsx'.format(c,b,bk), sheet_name='Sheet1')
    return df_final

In [361]:
%%time

b="RC"
c="VND"
bk = '30-90'

test_size = 30


data_regression= loc_bien(b,c,bk,wd_reg,all_var)

Y_variables=data_regression.columns[0]
X_variables=data_regression.columns[1:]
df_reg = pd.concat([data_regression,dummy_period_variables.iloc[:, -2:]], axis =1)
data_core_matrix=data_regression[X_variables].corr()
sign=(data_core_matrix.abs()<0.4).astype(int)
list_X_feature=[[i] for i in X_variables]
for n in range(2,len(X_variables)+1):
    check_n_1=[i for i in list_X_feature if len(i)==(n-1)]
    if len(check_n_1)==0:
        break
    list_X_feature_n=[]
    for a in check_n_1:
        list_X_feature_n+=[np.sort(a+[x]).tolist() for x in sign[sign[a].prod(1)==1].index]
            ###Chay model voi so bien <4 - tong so bien X
        list_X_feature_n= [i for i in list_X_feature_n if len(i)<5] 
    list_X_feature+=pd.DataFrame(list_X_feature_n).drop_duplicates().values.tolist() 
print(len(list_X_feature))
for i in list_X_feature:
    i += ['ss_dummy', 'covid_dummy' ]
#list_X_feature

#     """chạy loop mô hình"""
import threading
thread = []
for i in [Y_variables]: #depend on the name of z variable in excel file
    thread.append(threading.Thread(target=run_OLS_loop, args=(i,list_X_feature,test_size)))

for j in thread:
    j.start()
    j.join() 
    #np.seterr(invalid='ignore')

res_total = pd.read_excel('./total_models_{}_{}_WD_{}.xlsx'.format(c,b,bk))
res_total_final = write_model(res_total)
clear_output()

In [416]:
list_X_feature_copy = list_X_feature

In [411]:
list_test = list_X_feature[300:302]

In [403]:
list_test

[['GDP_lag_6m', 'GDP_per_change', 'ss_dummy', 'covid_dummy'],
 ['GDP_lag_6m', 'GDP_ma_3m', 'ss_dummy', 'covid_dummy']]

In [417]:
for i in list_X_feature_copy:
    check = i
    res =  [list(i) for j, i in groupby(check, lambda a: a.split('_')[0])]
    for z in res:
        if len(z) > 1:
            list_X_feature_copy.remove(i)
    break

In [421]:
len(list_X_feature_copy)

9125

In [409]:
res = [list(i) for j, i in groupby(list_test[0], lambda a: a.split('_')[0])]

In [410]:
res

[['GDP_lag_6m', 'GDP_per_change'], ['ss_dummy'], ['covid_dummy']]

In [396]:
for i in res:
    if len(i) > 1:
        list_test.remove(res)

In [397]:
list_test

[['GDP_lag_6m', 'GDP_per_change', 'ss_dummy', 'covid_dummy'],
 ['GDP_lag_6m', 'GDP_ma_3m', 'ss_dummy', 'covid_dummy']]

In [387]:
res

[['c'], ['o'], ['v'], ['i'], ['d'], ['_'], ['d'], ['u'], ['m', 'm'], ['y']]

In [ ]:
run_OLS(x_fea.copy(), y_fea,data=df_reg.copy(),poly_degree=1,return_plot=False,test_size=test_size,return_summary=False)

In [362]:
%%time

b="RC"
c="VND"
bk = '90-180'

test_size = 30


data_regression= loc_bien(b,c,bk,wd_reg,all_var)

Y_variables=data_regression.columns[0]
X_variables=data_regression.columns[1:]
df_reg = pd.concat([data_regression,dummy_period_variables.iloc[:, -2:]], axis =1)
data_core_matrix=data_regression[X_variables].corr()
sign=(data_core_matrix.abs()<0.4).astype(int)
list_X_feature=[[i] for i in X_variables]
for n in range(2,len(X_variables)+1):
    check_n_1=[i for i in list_X_feature if len(i)==(n-1)]
    if len(check_n_1)==0:
        break
    list_X_feature_n=[]
    for a in check_n_1:
        list_X_feature_n+=[np.sort(a+[x]).tolist() for x in sign[sign[a].prod(1)==1].index]
            ###Chay model voi so bien <4 - tong so bien X
        list_X_feature_n= [i for i in list_X_feature_n if len(i)<5] 
    list_X_feature+=pd.DataFrame(list_X_feature_n).drop_duplicates().values.tolist() 
print(len(list_X_feature))
for i in list_X_feature:
    i += ['ss_dummy', 'covid_dummy' ]
#list_X_feature

#     """chạy loop mô hình"""
import threading
thread = []
for i in [Y_variables]: #depend on the name of z variable in excel file
    thread.append(threading.Thread(target=run_OLS_loop, args=(i,list_X_feature,test_size)))

for j in thread:
    j.start()
    j.join() 
    #np.seterr(invalid='ignore')

res_total = pd.read_excel('./total_models_{}_{}_WD_{}.xlsx'.format(c,b,bk))
res_total_final = write_model(res_total)
clear_output()

Wall time: 1min 15s


In [368]:
%%time

b="RC"
c="VND"
bk = '180-365'

test_size = 30


data_regression= loc_bien(b,c,bk,wd_reg,all_var)

Y_variables=data_regression.columns[0]
X_variables=data_regression.columns[1:]
df_reg = pd.concat([data_regression,dummy_period_variables.iloc[:, -2:]], axis =1)
data_core_matrix=data_regression[X_variables].corr()
sign=(data_core_matrix.abs()<0.5).astype(int)
list_X_feature=[[i] for i in X_variables]
for n in range(2,len(X_variables)+1):
    check_n_1=[i for i in list_X_feature if len(i)==(n-1)]
    if len(check_n_1)==0:
        break
    list_X_feature_n=[]
    for a in check_n_1:
        list_X_feature_n+=[np.sort(a+[x]).tolist() for x in sign[sign[a].prod(1)==1].index]
            ###Chay model voi so bien <4 - tong so bien X
        list_X_feature_n= [i for i in list_X_feature_n if len(i)<5] 
    list_X_feature+=pd.DataFrame(list_X_feature_n).drop_duplicates().values.tolist() 
print(len(list_X_feature))
for i in list_X_feature:
    i += ['ss_dummy', 'covid_dummy' ]
#list_X_feature

#     """chạy loop mô hình"""
import threading
thread = []
for i in [Y_variables]: #depend on the name of z variable in excel file
    thread.append(threading.Thread(target=run_OLS_loop, args=(i,list_X_feature,test_size)))

for j in thread:
    j.start()
    j.join() 
    #np.seterr(invalid='ignore')

res_total = pd.read_excel('./total_models_{}_{}_WD_{}.xlsx'.format(c,b,bk))
res_total_final = write_model(res_total)
clear_output()

Wall time: 43min


In [369]:
%%time

b="RC"
c="VND"
bk = '365-730'

test_size = 30


data_regression= loc_bien(b,c,bk,wd_reg,all_var)

Y_variables=data_regression.columns[0]
X_variables=data_regression.columns[1:]
df_reg = pd.concat([data_regression,dummy_period_variables.iloc[:, -2:]], axis =1)
data_core_matrix=data_regression[X_variables].corr()
sign=(data_core_matrix.abs()<0.6).astype(int)
list_X_feature=[[i] for i in X_variables]
for n in range(2,len(X_variables)+1):
    check_n_1=[i for i in list_X_feature if len(i)==(n-1)]
    if len(check_n_1)==0:
        break
    list_X_feature_n=[]
    for a in check_n_1:
        list_X_feature_n+=[np.sort(a+[x]).tolist() for x in sign[sign[a].prod(1)==1].index]
            ###Chay model voi so bien <4 - tong so bien X
        list_X_feature_n= [i for i in list_X_feature_n if len(i)<5] 
    list_X_feature+=pd.DataFrame(list_X_feature_n).drop_duplicates().values.tolist() 
print(len(list_X_feature))
for i in list_X_feature:
    i += ['ss_dummy', 'covid_dummy' ]
#list_X_feature

#     """chạy loop mô hình"""
import threading
thread = []
for i in [Y_variables]: #depend on the name of z variable in excel file
    thread.append(threading.Thread(target=run_OLS_loop, args=(i,list_X_feature,test_size)))

for j in thread:
    j.start()
    j.join() 
    #np.seterr(invalid='ignore')

res_total = pd.read_excel('./total_models_{}_{}_WD_{}.xlsx'.format(c,b,bk))
res_total_final = write_model(res_total)
clear_output()

Wall time: 21min 34s
Compiler : 258 ms
Parser   : 1.07 s


In [60]:
%%time

b="RC"
c="VND"
tw = 730

test_size = 30


data_regression= loc_bien(b,c,tw,wd_reg,all_var)

Y_variables=data_regression.columns[0]
X_variables=data_regression.columns[1:]
df_reg = pd.concat([data_regression,dummy_period_variables.iloc[:, -2:]], axis =1)
data_core_matrix=data_regression[X_variables].corr()
sign=(data_core_matrix.abs()<0.4).astype(int)
list_X_feature=[[i] for i in X_variables]
for n in range(2,len(X_variables)+1):
    check_n_1=[i for i in list_X_feature if len(i)==(n-1)]
    if len(check_n_1)==0:
        break
    list_X_feature_n=[]
    for a in check_n_1:
        list_X_feature_n+=[np.sort(a+[x]).tolist() for x in sign[sign[a].prod(1)==1].index]
            ###Chay model voi so bien <4 - tong so bien X
        list_X_feature_n= [i for i in list_X_feature_n if len(i)<4] 
    list_X_feature+=pd.DataFrame(list_X_feature_n).drop_duplicates().values.tolist() 
print(len(list_X_feature))
for i in list_X_feature:
    i += ['ss_dummy', 'covid_dummy' ]
#list_X_feature

#     """chạy loop mô hình"""
import threading
thread = []
for i in [Y_variables]: #depend on the name of z variable in excel file
    thread.append(threading.Thread(target=run_OLS_loop, args=(i,list_X_feature,test_size)))

for j in thread:
    j.start()
    j.join() 
    #np.seterr(invalid='ignore')

res_total = pd.read_excel('./total_models_{}_{}_WD_{:03}.xlsx'.format(c,b,tw))
res_total_final = write_model(res_total)
clear_output()

Wall time: 51 s
Parser   : 106 ms


In [58]:
%%time

b="RC"
c="VND"
tw = 1095

# test_size = 30


# data_regression= loc_bien(b,c,tw,wd_reg,all_var)

# Y_variables=data_regression.columns[0]
# X_variables=data_regression.columns[1:]
# df_reg = pd.concat([data_regression,dummy_period_variables.iloc[:, -2:]], axis =1)
# data_core_matrix=data_regression[X_variables].corr()
# sign=(data_core_matrix.abs()<0.4).astype(int)
# list_X_feature=[[i] for i in X_variables]
# for n in range(2,len(X_variables)+1):
#     check_n_1=[i for i in list_X_feature if len(i)==(n-1)]
#     if len(check_n_1)==0:
#         break
#     list_X_feature_n=[]
#     for a in check_n_1:
#         list_X_feature_n+=[np.sort(a+[x]).tolist() for x in sign[sign[a].prod(1)==1].index]
#             ###Chay model voi so bien <4 - tong so bien X
#         list_X_feature_n= [i for i in list_X_feature_n if len(i)<4] 
#     list_X_feature+=pd.DataFrame(list_X_feature_n).drop_duplicates().values.tolist() 
# print(len(list_X_feature))
# for i in list_X_feature:
#     i += ['ss_dummy', 'covid_dummy' ]
# #list_X_feature

# #     """chạy loop mô hình"""
# import threading
# thread = []
# for i in [Y_variables]: #depend on the name of z variable in excel file
#     thread.append(threading.Thread(target=run_OLS_loop, args=(i,list_X_feature,test_size)))

# for j in thread:
#     j.start()
#     j.join() 
#     #np.seterr(invalid='ignore')

res_total = pd.read_excel('./total_models_{}_{}_WD_{:03}.xlsx'.format(c,b,tw))
res_total_final = write_model(res_total)
clear_output()

Wall time: 2.77 s


In [59]:
%%time

b="RC"
c="VND"
tw = 1460

# test_size = 30


# data_regression= loc_bien(b,c,tw,wd_reg,all_var)

# Y_variables=data_regression.columns[0]
# X_variables=data_regression.columns[1:]
# df_reg = pd.concat([data_regression,dummy_period_variables.iloc[:, -2:]], axis =1)
# data_core_matrix=data_regression[X_variables].corr()
# sign=(data_core_matrix.abs()<0.4).astype(int)
# list_X_feature=[[i] for i in X_variables]
# for n in range(2,len(X_variables)+1):
#     check_n_1=[i for i in list_X_feature if len(i)==(n-1)]
#     if len(check_n_1)==0:
#         break
#     list_X_feature_n=[]
#     for a in check_n_1:
#         list_X_feature_n+=[np.sort(a+[x]).tolist() for x in sign[sign[a].prod(1)==1].index]
#             ###Chay model voi so bien <4 - tong so bien X
#         list_X_feature_n= [i for i in list_X_feature_n if len(i)<4] 
#     list_X_feature+=pd.DataFrame(list_X_feature_n).drop_duplicates().values.tolist() 
# print(len(list_X_feature))
# for i in list_X_feature:
#     i += ['ss_dummy', 'covid_dummy' ]
# #list_X_feature

# #     """chạy loop mô hình"""
# import threading
# thread = []
# for i in [Y_variables]: #depend on the name of z variable in excel file
#     thread.append(threading.Thread(target=run_OLS_loop, args=(i,list_X_feature,test_size)))

# for j in thread:
#     j.start()
#     j.join() 
#     #np.seterr(invalid='ignore')

res_total = pd.read_excel('./total_models_{}_{}_WD_{:03}.xlsx'.format(c,b,tw))
res_total_final = write_model(res_total)
clear_output()

Wall time: 328 ms


In [50]:
all_var.to_excel("all_var_irrbb.xlsx")

#### Lọc model

In [ ]:
# def X_sign(list_x):
#     res=[]
#     IR_names=all_var.columns
#     for i in list_x:
#         if isinstance(i,(int,float)):
#             res.append(np.sign(i))
#         elif i =="const":
#             res.append(0)
#         elif i in IR_names:
#             res.append(-1)
#         elif i in "ss_dummy":
#             res.append(1)
#         else: # covid dummy
#             res.append(0)
#     return res

# def filter_true_model(model):
#     l1=model.Coef.map(X_sign).tolist()
#     l2=model.X_features.map(X_sign).tolist()
#     res=[]
#     for i in range(len(l1)):
#         check=[(a*b) for a,b in zip(l1[i],l2[i])]
#         res.append((-1 not in check))
#     return res

# def true_model(b,c):
#     model=pd.read_pickle("Regression/check_model_102021_{}_{}.pickle".format(c,b))
#     res=model.loc[filter_true_model(model)].sort_values(["RMSE","R_2"],ascending=[True,False])
#     res.drop_duplicates(['R_2','RMSE'],inplace=True)
#     res.to_excel("Regression/true_model_{}_{}.xlsx".format(c,b))
#     return res

In [ ]:
# for b,c in [("RC","VND")]:
#     true_model(b=b,c=c)

#### Backtest

In [ ]:
def plot_backtest_mix(b,c,tw,df,MB_Core_all=MB_Core_all):
    df=df.copy()
    df["WDss"]=(df["WD"]*df.index.isin(ss_day_t0)).replace(0,np.nan)
    df["WDst"]=(df["WD"]*df.index.isin(st_msb )).replace(0,np.nan)
    t0=MB_Core_all[[(c,b,"sum_000")]].reindex(df.index).values
    df2=df*t0
    df2["T0"]=t0
    return df2

In [ ]:
b="RC"
c="VND"
tw=30

ir_names= ['MSB_G1_Long_Term'] #[  'MSB_G2_Medium_Term', 'MSB_Long_Term']
dummy_names=  ["covid_dummy","ss_dummy"] #
# return_to = None                 # detail plot_reg plot_mix backtest_mix
run_OLS(b=b,c=c,tw=tw,ir_names=ir_names,dummy_names=dummy_names,wd_reg=wd_reg,return_to="backtest_mix",test_size=test_size,save_fig=True,per=per)

In [ ]:
def concat_data_plot(per,reg):
    return pd.concat([per,reg[[i for i in reg.columns if i not in per.columns]]],axis=1)

In [ ]:
def Backtest_Timeseries_mix(b,c,tw,df):
    
    uni,div_uni=("Millions",1000000) if c=="USD" else ("Billions",1000000000)
    df=df/div_uni
    max_y_axis=df.drop(["T0","TW","Contractual_WD"],axis=1).max().max()*1.3
    x = df.index

    
    # plot time count cif
    fig, ax1 = plt.subplots(1,1, figsize=(18, 10),dpi=100)

    ax1.set_title("BACKTEST RÚT TIỀN DANH MỤC {} {} TIME-WINDOW {} ngày".format(b,c,tw))

    ax1.grid(alpha=0.2)
    
    cols=["WD","WDss","WDst","Normal","Seasonal","Stress","mix_regression"]
    col_labels=["Giá trị rút tại tại {} ngày sau tại thời điểm T0".format(tw),
               "Giá trị rút tại tại {} ngày sau tại thời điểm T0 là giai đoạn mùa vụ".format(tw),
                "Giá trị rút tại tại {} ngày sau tại thời điểm T0 là giai đoạn căng thẳng".format(tw),
                "Ước lượng percentile rút thông thường","Ước lượng percentile rút mùa vụ","Ước lượng percentile rút căng thẳng",
                "Ước lượng hồi quy sau hiệu chỉnh",
               ]
    Colors=["grey","green","orange","brown","blue","olivedrab","red"]
    
    for co,la,color in zip(cols,col_labels,Colors):
        ls,alp=("-",0.7) if co in ["WD", "WDss","WDst","mix_regression"] else ("--",1)
        li=1.5 if co=="mix_regression" else 1
        try:
            ax1.plot(x,gaussian_filter1d(df[co], sigma=1),label=la,color=color,ls=ls,alpha=alp,linewidth=li)

        except:
            print("lỗi ",co)
            pass
    ax1.fill_between(x, 0, gaussian_filter1d(df["Contractual_WD"], sigma=1),label="Giá trị hợp đồng đáo hạn trong timewindow",color="violet", alpha=0.1)
    ax1.fill_between(x, 0, gaussian_filter1d(df["WD"], sigma=1), alpha=0.2)


    #plot normal period
    end_train_day_tw=end_train_day-pd.Timedelta(days=tw)
    ax1.vlines(x=x[(x<=end_train_day_tw) & (x.isin(ss_day_t0)==False)][-365], ymin=0, ymax=max_y_axis,ls="--")
    ax1.vlines(x=x[(x<=end_train_day_tw) & (x.isin(ss_day_t0)==False)][-1], ymin=0, ymax=max_y_axis,ls="--")
    ax1.annotate(text="Normal Period",xy=(x[x.isin(ss_day_t0)==False][-200],max_y_axis*0.8))
    
    ax1.legend(loc="upper left",ncol=3)
    ax1.set_ylim(0, max_y_axis)

    ax1.set_ylabel(uni)

    ax1.set_xlabel('Begin date')
    dayloc = mdates.MonthLocator([3,6,9,12])
    ax1.xaxis.set_major_locator(dayloc)
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    fig.autofmt_xdate()
#     plt.grid(alpha=0.4)
    plt.show()
    return fig

In [ ]:
def plot_backtest_violation_reg(df,b,c,tw):
    
    df=df[df.index<=(end_train_day-pd.Timedelta(days=tw))].rename(columns={"mix_regression":"Estimate"}).copy()
    uni,div_uni=("Millions",1000000) if c=="USD" else ("Billions",1000000000)
    list_fig=[]
    list_condition=["All"] if b=="L" else ["Normal","Seasonal","All"]
    for condition in list_condition:
        fig, axs = plt.subplots(1, 2, figsize=(22.5, 3),dpi=68)
        if condition=="Normal":
            df2=df[df.index.isin(ss_day_t0)==False].iloc[-365:].copy()
        elif condition=="Seasonal":
            df2=df[df.index.isin(ss_day_t0)].copy()
        else:
            df2 = df.copy()
            
        df2["number_vio"]=(df2["WD"]>df2["Estimate"]).astype(int)
        df2["amount_vio"]=(df2["WD"]-df2["Estimate"])*df2["number_vio"]
        df2["per_amount_vio"]=df2["amount_vio"]/df2["T0"]
    #         global df2_month
        df2_month=df2.groupby(pd.Grouper(freq='MS')).agg({"number_vio":"sum","amount_vio":"mean","per_amount_vio":["max","mean"]}).applymap(lambda x: x if x>0 else np.nan)
        df2_month.columns=df2_month.columns.map(lambda x: x[0]+"_"+x[1])
        x=df2_month.index

        ax_1,ax_2 = axs[0],axs[1]
        #count violation
        ax_1.grid(alpha=0.4)
        ax_1.bar(x,df2_month.number_vio_sum.fillna(0),width=15)
        ax_1.set_ylabel('Số vi phạm')
        ax_1.set_title("BACKTEST SỐ LẦN VI PHẠM ƯỚC LƯỢNG HỒI QUY DANH MỤC {} {} TIME-WINDOW {} NGÀY ({})".format(b,c,tw,condition))
        for i in df2_month.dropna().index:
            ax_1.annotate(text=df2_month.loc[i,"number_vio_sum"].astype(int),xy=(i,df2_month.loc[i,"number_vio_sum"]),  va='center',xytext=(-3,7), textcoords='offset points')

        dayloc = mdates.MonthLocator([3,6,9,12])
        ax_1.xaxis.set_major_locator(dayloc)
        ax_1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
        try:
            ax_1.set_ylim(0, df2_month.number_vio_sum.max()*1.2)
        except:
            pass
        ax_1.set_xlabel('Begin date')

        # balance of violation
        ax_2.set_title("BACKTEST MỨC ĐỘ VI PHẠM ƯỚC LƯỢNG HỒI QUY DANH MỤC {} {} TIME-WINDOW {} NGÀY ({})".format(b,c,tw,condition))
        ax_2.grid(alpha=0.4)
        ax_2.fill_between(x, 0, df2_month['per_amount_vio_max'].fillna(0)*100, alpha=0.4,color="orange",label="Tỷ lệ (%) vi phạm lớn nhất")
        ax_2.fill_between(x, 0, df2_month['per_amount_vio_mean'].fillna(0)*100, alpha=1,color="orange",label="Tỷ lệ (%) vi phạm trung bình")
        ax_2.set_ylabel('%')
        ax_2.set_xlabel('Begin date')

        ax2_2=ax_2.twinx()
        ax2_2.bar(x,df2_month["amount_vio_mean"].fillna(0)/div_uni,width=15,label="Giá trị vi phạm trung bình")
        ax2_2.set_ylabel(uni)

        ax_2.legend(loc="upper left",ncol=2)
        ax2_2.legend(loc="upper right")

        ax_2.xaxis.set_major_locator(dayloc)
        ax_2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
        try:
            ax_2.set_ylim(0, df2_month['per_amount_vio_max'].max()*140)
            ax2_2.set_ylim(0, df2_month['amount_vio_mean'].max()/div_uni*1.4)
        except:
            pass
        fig.autofmt_xdate()
        plt.show()
        list_fig.append(fig)
    return list_fig,list_condition

## Backup toàn bộ

In [2]:
''' phân loại số tài khoản theo mã sản phẩm '''
def cal_rate(date,table,func=np.nansum):
    df = table[["TYPE",'ACCTNO']].groupby(["TYPE"]).count()
    if df.columns.nlevels==2:
        df=df.melt(ignore_index=False,col_level=1,var_name=["func"],value_name=date).set_index("func",append=True)
    else:
        df.columns=[date]
    return df.T


Result=pd.DataFrame()
for date in list_date:
    Result_day=pd.DataFrame()
    for i in range(1):
        print(date.date(),list_tw1[i],list_tw2[i],end="\r")
        all_mat_bucket=return_df_tw2(date,table=data,method_mat="ALL_{}_{}".format(0,30))
        date_tw1=date+pd.Timedelta("{}D".format(0)) # date+tw1
        date_tw2=date+pd.Timedelta("{}D".format(30)) # date+tw2
        all_tw1=return_df_tw2(date_tw1,table=data,method_mat="ALL_0_0") # danh mục tồn tại tại ngày date + tw1
        all_tw2=return_df_tw2(date_tw2,table=data,method_mat="ALL_0_0") # danh mục tồn tại tại ngày date + tw2
        all_wit_bucket = all_mat_bucket[all_mat_bucket.ACCTNO.isin(all_tw1.ACCTNO)][~all_mat_bucket.ACCTNO.isin(all_tw2.ACCTNO)]
        all_wit_bucket_sum = cal_rate(date,table=all_wit_bucket,func=np.nansum)
        Result_day=pd.concat([Result_day,all_wit_bucket_sum],axis=1)
        clear_output()
    Result =Result.append(Result_day)

Result
# Result.to_excel(r'D:\IRRBB 2022\output_T10_2022\backup\30-90.xlsx')

NameError: name 'np' is not defined

In [ ]:
''' phân loại số tài khoản dựa trên kì hạn hợp đồng '''
## chạy classify dựa trên classify_term (kì hạn hợp đồng)
def cal_rate(date,table,func=np.nansum):
    df = table[["Classify_term",'ACCTNO']].groupby(["Classify_term"]).count()
    if df.columns.nlevels==2:
        df=df.melt(ignore_index=False,col_level=1,var_name=["func"],value_name=date).set_index("func",append=True)
    else:
        df.columns=[date]
    return df.T


Result=pd.DataFrame()
for date in list_date[:2]:
    Result_day=pd.DataFrame()
    for i in range(1):
        print(date.date(),list_tw1[i],list_tw2[i],end="\r")
        all_mat_bucket=return_df_tw2(date,table=data,method_mat="ALL_{}_{}".format(30,90))
        date_tw1=date+pd.Timedelta("{}D".format(30)) # date+tw1
        date_tw2=date+pd.Timedelta("{}D".format(90)) # date+tw2
        all_tw1=return_df_tw2(date_tw1,table=data,method_mat="ALL_0_0") # danh mục tồn tại tại ngày date + tw1
        all_tw2=return_df_tw2(date_tw2,table=data,method_mat="ALL_0_0") # danh mục tồn tại tại ngày date + tw2
        all_wit_bucket = all_mat_bucket[all_mat_bucket.ACCTNO.isin(all_tw1.ACCTNO)][~all_mat_bucket.ACCTNO.isin(all_tw2.ACCTNO)]
        all_wit_bucket_sum = cal_rate(date,table=all_wit_bucket,func=np.nansum)
        Result_day=pd.concat([Result_day,all_wit_bucket_sum],axis=1)
        clear_output()
    Result =Result.append(Result_day)

Result
# Result.to_excel(r'D:\IRRBB 2022\output_T10_2022\backup\30-90.xlsx')

In [ ]:
''' tính số lượng account rút trước hạn '''

def return_df_tw2(date,table=data,danhmuc=[None,None],method_mat="ALL_0_1"):
    tw1=int(method_mat.split("_")[1])
    tw2=int(method_mat.split("_")[-1])
    method = method_mat.split("_")[0]
    c_mat_1 = ((table.MAT_DATE>(date+pd.Timedelta(str(tw2)+"D")))\
               if ("ALL" in method) else False)  
    c_wit_cond1 = ((table.MAT_DATE>(date+pd.Timedelta(str(tw2)+"D"))) if ("wbetween" in method) else False)
    c_wit_1 = ((table.CHECK_TO_DATE>=(date+pd.Timedelta(str(tw1)+"D")) if (tw1 == 0) else table.CHECK_TO_DATE>=(date+pd.Timedelta(str(tw1)+"D")))\
               if ("wbetween" in method) else False)
    c_wit_2 = ((((table.CHECK_TO_DATE )<(date+pd.Timedelta(str(tw2)+"D"))))  if ("wbetween" in method) else False) 
    c_wit_between = (c_wit_cond1 & c_wit_1 &  c_wit_2)
    return table [(table.CHECK_FROM_DATE<=date) & (table.CHECK_TO_DATE>=date) & ( c_mat_1 | c_wit_between) ].copy()


def calculate_sum_balance(date,t,table=data,key="ALL"):        
    df = table[["CURTYP","bank","ACCTNO"]].groupby(["CURTYP","bank"]).count()
    if df.columns.nlevels==2:
        df=df.melt(ignore_index=False,col_level=1,var_name=["func"],value_name=date).set_index("func",append=True)
    else:
        df.columns=[date]
    df[["KIND","Bucket"]]=[key,t]
    df.set_index(["KIND","Bucket"],append=True,inplace=True)
    return df.T

def Nan_to_tw(table,level=3):
    table.fillna(0,inplace=True)
    #for i in table.columns:
        #if int(i[level].split('-')[1].split('D')[0])>0: table.loc[table.index[-(int(i[level].split('-')[1].split('D')[0])>0):],i]=np.nan

Result=pd.DataFrame()
list_tw1=[0,30,90,180,365,730,1095,1460,1825]
list_tw2=[30,90,180,365,730,1095,1460,1825,2190]
for date in list_date:
    Result_day=pd.DataFrame()
    for i in range(9):
        print(date.date(),list_tw1[i],list_tw2[i],end="\r")
        all_mat_bucket=return_df_tw2(date,table=data,method_mat="ALL_{}_{}".format(list_tw1[i],list_tw2[i])) # trả về bảng có mat > date + tw2
        all_mat_bucket_sum=calculate_sum_balance(date,t = '{}-{}'.format(list_tw1[i],list_tw2[i]),table=all_mat_bucket,key="ALL") # tính tổng giá trị all_mat_bucket
        Result_day=pd.concat([Result_day,all_mat_bucket_sum],axis=1)
        date_tw1=date+pd.Timedelta("{}D".format(list_tw1[i])) # date+tw1
        date_tw2=date+pd.Timedelta("{}D".format(list_tw2[i])) # date+tw2
        all_tw1=return_df_tw2(date_tw1,table=data,method_mat="ALL_0_0") # danh mục tồn tại tại ngày date + tw1
        all_tw2=return_df_tw2(date_tw2,table=data,method_mat="ALL_0_0") # danh mục tồn tại tại ngày date + tw2
        all_wit_bucket = all_mat_bucket[all_mat_bucket.ACCTNO.isin(all_tw1.ACCTNO)][~all_mat_bucket.ACCTNO.isin(all_tw2.ACCTNO)]
        all_wit_bucket_sum=calculate_sum_balance(date,t ='{}-{}'.format(list_tw1[i],list_tw2[i]),table=all_wit_bucket,key="PRE") #tính bảng có account RTHall_mat_bucket        
        Result_day=pd.concat([Result_day,all_wit_bucket_sum],axis=1)
        clear_output()
    Result =Result.append(Result_day)
    
Nan_to_tw(Result)
Result=Result.sort_index(axis=1, level = [0,1,2], sort_remaining = False)
BWex=modify_columns_name(Result.iloc[:,Result.columns.get_level_values(1)=="RC"],1,"BWex",axis=1).add(modify_columns_name(Result.iloc[:,Result.columns.get_level_values(1)=="LSB"],1,"BWex",axis=1),fill_value=0)
Result_acc=pd.concat([Result,BWex],axis=1).sort_index(axis=1, level = [0], sort_remaining = False)
#export
Result_acc.to_pickle(output_folder+r"Result_acc.pickle")
Result_acc


In [ ]:
''' tính số lượng account rút trước hạn
phân chia theo số ngày còn lại của hợp đồng '''
# Tính số ngày còn lại của hợp đồng đối với những tk rút trước hạn
def term_dleft(x):
    if x <=30: x='0-1M'
    elif x <=90: x='1-3M'
    elif x <=180: x ='3-6M'
    elif x <=365: x ='6M-1Y'
    elif x <= 730: x ='1Y-2Y'
    elif x <= 1095: x = '2-3Y'
    else: x='>3Y'
    return x

def return_df_tw2(date,table=data,danhmuc=[None,None],method_mat="ALL_0_1"):
    # Tw1, Tw2 là hai giá trị chặn trong Bucket muốn xét
    # Method = ALL => xác định balance có MAT > TW2, Method = wBetween => Xác định withdrawal between Tw1 và TW2
    tw1=int(method_mat.split("_")[1])
    tw2=int(method_mat.split("_")[-1])
    method = method_mat.split("_")[0]
    c_mat_1 = ((table.MAT_DATE>(date+pd.Timedelta(str(tw2)+"D")))\
               if ("ALL" in method) else False)  
    #các điều kiện xác định account rút trước hạn (mat>tw2 & rút trước hạn trong bucket đang xét)
    c_wit_cond1 = ((table.MAT_DATE>(date+pd.Timedelta(str(tw2)+"D"))) if ("wbetween" in method) else False)
    c_wit_1 = ((table.CHECK_TO_DATE>=(date+pd.Timedelta(str(tw1)+"D")) if (tw1 == 0) else table.CHECK_TO_DATE>=(date+pd.Timedelta(str(tw1)+"D")))\
               if ("wbetween" in method) else False)
    c_wit_2 = ((((table.CHECK_TO_DATE )<(date+pd.Timedelta(str(tw2)+"D"))))  if ("wbetween" in method) else False) 
    #c_wit_2 = (((table.CHECK_TO_DATE + pd.Timedelta(str(1)+"D"))<(date+pd.Timedelta(str(tw2)+"D"))) if ("wbetween" in method) else False)  
    c_wit_between = (c_wit_cond1 & c_wit_1 &  c_wit_2)
    
    return table [(table.CHECK_FROM_DATE<=date) & (table.CHECK_TO_DATE>=date) & ( c_mat_1 | c_wit_between) ].copy()

def cal_rate(date,table,func=np.nansum):
    df = table[["Dleft_term",'ACCTNO']].groupby(["Dleft_term"]).count()
    if df.columns.nlevels==2:
        df=df.melt(ignore_index=False,col_level=1,var_name=["func"],value_name=date).set_index("func",append=True)
    else:
        df.columns=[date]
    return df.T

list_tw1=[30]
list_tw2=[90]
Result=pd.DataFrame()
for date in list_date:
    Result_day=pd.DataFrame()
    for i in range(1):
        print(date.date(),list_tw1[i],list_tw2[i],end="\r")
        all_mat_bucket=return_df_tw2(date,table=data,method_mat="ALL_{}_{}".format(30,90))
        date_tw1=date+pd.Timedelta("{}D".format(30)) # date+tw1
        date_tw2=date+pd.Timedelta("{}D".format(90)) # date+tw2
        all_tw1=return_df_tw2(date_tw1,table=data,method_mat="ALL_0_0") # danh mục tồn tại tại ngày date + tw1
        all_tw2=return_df_tw2(date_tw2,table=data,method_mat="ALL_0_0") # danh mục tồn tại tại ngày date + tw2
        all_wit_bucket = all_mat_bucket[all_mat_bucket.ACCTNO.isin(all_tw1.ACCTNO)][~all_mat_bucket.ACCTNO.isin(all_tw2.ACCTNO)]
        all_wit_bucket = all_mat_bucket[all_mat_bucket.ACCTNO.isin(all_tw1.ACCTNO)][~all_mat_bucket.ACCTNO.isin(all_tw2.ACCTNO)]
        all_wit_bucket['Dleft'] = (all_wit_bucket.MAT_DATE - all_wit_bucket.CHECK_TO_DATE)// pd.Timedelta ('1D')
        all_wit_bucket['Dleft_term'] = all_wit_bucket.Dleft.map(term_dleft)
        all_wit_bucket_sum = cal_rate(date,table=all_wit_bucket,func=np.nansum)
        Result_day=pd.concat([Result_day,all_wit_bucket_sum],axis=1)
        clear_output()
    Result =Result.append(Result_day)

Result
# Result.to_excel(r'D:\IRRBB 2022\output_T10_2022\backup\30-90.xlsx')

In [ ]:
''' USD/theo mã sản phẩm ứng vốn '''
'''
CHẠY USD THEO LIST TK ỨNG VỐN 
'''
# Lọc tài khoản ứng vốn
# df = data[data.CURTYP == 'USD']
# tk_ungvon = pd.read_excel (r'D:\IRRBB 2022\output_T10_2022\TK ỨNG VỐN.xlsx')
# df_ungvon = data[data.TYPE.isin(tk_ungvon.ACCTNO)]

# Define hàm xác định tài khoản và tính toán
def return_df_tw2(date,table=data):
    return table [(table.CHECK_FROM_DATE<=date) & (table.CHECK_TO_DATE>=date) & (table.MAT_DATE>=table.CHECK_TO_DATE) ].copy()

def cal_rate(date,table,func=np.nansum):
    df = table[['bank',"TYPE",'CURRENT_BALANCE']].groupby(['bank',"TYPE"]).agg(func)
    if df.columns.nlevels==2:
        df=df.melt(ignore_index=False,col_level=1,var_name=["func"],value_name=date).set_index("func",append=True)
    else:
        df.columns=[date]
    return df.T

Result=pd.DataFrame()
for date in list_date:
    Result_day=pd.DataFrame()
    for i in range(1):
        print(date.date(),list_tw1[i],list_tw2[i],end="\r")
        all_mat_bucket=return_df_tw2(date,table=df_ungvon)
        all_mat_bucket_sum=cal_rate(date,table=all_mat_bucket) 
        Result_day=pd.concat([Result_day,all_mat_bucket_sum],axis=1)
        clear_output()
    Result =Result.append(Result_day)
Result=Result.sort_index(axis=1, level = [0], sort_remaining = False)
Result

# Result.to_excel(r'D:\IRRBB 2022\output_T10_2022\backup\30-90.xlsx')

In [ ]:
''' VND/ mã sản phẩm (nói chung) '''
# Define hàm xác định tài khoản và tính toán
def return_df_tw2(date,table=data):
    return table [(table.CHECK_FROM_DATE<=date) & (table.CHECK_TO_DATE>=date) & (table.MAT_DATE>=table.CHECK_TO_DATE) ].copy()

def cal_rate(date,table,func=np.nansum):
    df = table[['bank',"TYPE",'CURRENT_BALANCE']].groupby(['bank',"TYPE"]).agg(func)
    if df.columns.nlevels==2:
        df=df.melt(ignore_index=False,col_level=1,var_name=["func"],value_name=date).set_index("func",append=True)
    else:
        df.columns=[date]
    return df.T

Result=pd.DataFrame()
for date in list_date:
    Result_day=pd.DataFrame()
    for i in range(1):
        print(date.date(),end="\r")
        all_mat_bucket=return_df_tw2(date,table=data)
        all_mat_bucket_sum=cal_rate(date,table=all_mat_bucket) 
        Result_day=pd.concat([Result_day,all_mat_bucket_sum],axis=1)
        clear_output()
    Result =Result.append(Result_day)
Result=Result.sort_index(axis=1, level = [0], sort_remaining = False)
Result

# Result.to_excel(r'D:\IRRBB 2022\output_T10_2022\backup\30-90.xlsx')

# Tính lãi suất huy động

In [ ]:
# Tạo cột current balance theo rate
data['cbal_rate']=data['CURRENT_BALANCE'] *data['RATE']

# Define các hàm cần thiết
def return_df_tw2(date,table=data):
    return table [(table.CHECK_FROM_DATE<=date) & (table.CHECK_TO_DATE>=date) &  (table.MAT_DATE >= table.CHECK_TO_DATE)  ].copy()

def cal_rate(date,table,func=np.nansum):
    df=table.groupby(['bank','CURTYP','Classify_term']).agg({'cbal_rate':'sum','CURRENT_BALANCE':'sum'})\
                    .eval('avg_rate = cbal_rate / CURRENT_BALANCE').loc[:,'avg_rate'].to_frame()
    if df.columns.nlevels==2:
        df=df.melt(ignore_index=False,col_level=1,var_name=["func"],value_name=date).set_index("func",append=True)
    else:
        df.columns=[date]
    return df.T


start_date=pd.Timestamp('2016-01-01')
end_date=pd.Timestamp('2022-11-30')
list_date=pd.date_range(start=start_date,end=end_date).tolist()
Result=pd.DataFrame()
for date in list_date:
    Result_day=pd.DataFrame()
    for i in range(4):
        print(date.date(),list_tw1[i],list_tw2[i],end="\r")
        all_bucket=return_df_tw2(date,table=data) 
        all_bucket_sum=cal_rate(date,table=all_bucket) 
        clear_output()
    Result= Result.append(all_bucket_sum)
# Result.to_pickle (r'D:\IRRBB 2022\Lãi suất huy động\lshd.pickle')
# Result.to_excel (r'D:\IRRBB 2022\Lãi suất huy động\lshd.xlsx')
Result